## Example Queries and Usage Guide

### Example Skills-based Search Queries:
- `Python,Django,PostgreSQL`
- `React,JavaScript,TypeScript`
- `Machine Learning,Python,TensorFlow`
- `AWS,Docker,Kubernetes`

### Example Smart Search Queries:
- "Find senior Python developers with 5+ years experience"
- "Show me frontend developers who know React and Vue"
- "Search for data scientists with machine learning experience"
- "Find DevOps engineers who worked with AWS and Docker"
- "Show me candidates who worked at Google or Microsoft"

### Performance Testing Scenarios:
- Single user search performance
- Multiple concurrent users (5, 10, 20)
- High-frequency search patterns
- Large result set handling

### Monitoring and Alerting:
Use this notebook to:
- Monitor API performance over time
- Set up automated testing pipelines
- Create performance baselines
- Detect performance regressions
- Validate new feature deployments

---

**Note**: Make sure the API server is running on `localhost:8000` before executing this notebook. Update the `base_url` in the APIClient initialization if your server runs on a different address.

**Requirements**:
- API server running on localhost:8000
- Python packages: httpx, matplotlib, seaborn, pandas, asyncio
- Valid user credentials for authentication testing

In [1]:
async def generate_test_summary_and_cleanup():
    """Generate comprehensive test summary report and cleanup"""
    print("📋 COMPREHENSIVE TEST SUMMARY REPORT")
    print("=" * 50)
    print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"API Base URL: {api_client.base_url}")
    print()
    
    # Authentication summary
    print("🔐 AUTHENTICATION TESTING")
    print("-" * 30)
    print(f"Authentication Status: {'✅ PASSED' if auth_success else '❌ FAILED'}")
    if auth_success:
        print(f"Token Acquired: ✅ Yes")
    print()
    
    # Performance summary
    print("⚡ PERFORMANCE TESTING")
    print("-" * 30)
    
    performance_summary = performance_tracker.summary()
    if 'total_requests' in performance_summary:
        print(f"Total Requests: {performance_summary['total_requests']}")
        print(f"Success Rate: {performance_summary['success_rate']:.1f}%")
        print(f"Average Response Time: {performance_summary['avg_response_time']:.3f}s")
        print(f"95th Percentile Response Time: {performance_summary['p95_response_time']:.3f}s")
        print(f"Endpoints Tested: {performance_summary['endpoints_tested']}")
    else:
        print("No performance metrics available")
    
    if performance_results:
        print("\nConcurrency Test Results:")
        for concurrency, results in performance_results.items():
            print(f"  {concurrency} concurrent requests:")
            print(f"    Success Rate: {results['success_rate']:.1f}%")
            print(f"    Throughput: {results['throughput']:.1f} req/s")
            print(f"    Avg Response Time: {results['avg_response_time']:.3f}s")
    print()
    
    # Search functionality summary
    print("🔍 SEARCH FUNCTIONALITY")
    print("-" * 30)
    
    if search_results:
        for search_type, data in search_results.items():
            if isinstance(data, dict):
                if 'total_results' in data:
                    print(f"{search_type}: {data['total_results']} results found")
                elif search_type == 'filters':
                    companies_count = len(data.get('companies', []))
                    skills_count = len(data.get('skills', []))
                    departments_count = len(data.get('departments', []))
                    print(f"Available Filters: {companies_count} companies, {skills_count} skills, {departments_count} departments")
    else:
        print("No search results available")
    print()
    
    # Export test results
    try:
        metrics_df = performance_tracker.get_dataframe()
        if not metrics_df.empty:
            output_dir = Path('test_results')
            output_dir.mkdir(exist_ok=True)
            
            csv_file = output_dir / f"api_test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            metrics_df.to_csv(csv_file, index=False)
            print(f"✅ Performance metrics exported to: {csv_file}")
            
            # Export summary JSON
            summary_data = {
                'test_timestamp': datetime.now().isoformat(),
                'api_base_url': api_client.base_url,
                'authentication_success': auth_success,
                'performance_summary': performance_tracker.summary(),
                'concurrency_results': performance_results,
            }
            
            json_file = output_dir / f"api_test_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            with open(json_file, 'w') as f:
                json.dump(summary_data, f, indent=2, default=str)
            print(f"✅ Test summary exported to: {json_file}")
        else:
            print("⚠️ No performance metrics to export")
    except Exception as e:
        print(f"❌ Export failed: {e}")
    
    # Close HTTP client
    try:
        await api_client.close()
        print("✅ API client closed successfully")
    except Exception as e:
        print(f"⚠️ Error closing API client: {e}")
    
    print("\n🎉 Testing completed successfully!")
    print("\nNext Steps:")
    print("1. Review the performance visualizations above")
    print("2. Check exported CSV and JSON files in test_results/ directory")
    print("3. Address any recommendations from the summary report")
    print("4. Run this notebook regularly to monitor API performance")

# Generate final report and cleanup
await generate_test_summary_and_cleanup()

📋 COMPREHENSIVE TEST SUMMARY REPORT


NameError: name 'datetime' is not defined

## 6. Summary Report and Cleanup

Generate a comprehensive summary of all test results and clean up resources.

In [ ]:
def create_performance_visualizations(performance_results, performance_tracker):
    """Create performance visualization charts"""
    print("📊 Creating Performance Visualizations\n")
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('API Performance Analysis', fontsize=16, fontweight='bold')
    
    # 1. Response time by concurrency level
    if performance_results:
        concurrency_levels = list(performance_results.keys())
        avg_response_times = [performance_results[c]['avg_response_time'] for c in concurrency_levels]
        max_response_times = [performance_results[c]['max_response_time'] for c in concurrency_levels]
        
        axes[0, 0].plot(concurrency_levels, avg_response_times, 'o-', label='Average', linewidth=2, markersize=8)
        axes[0, 0].plot(concurrency_levels, max_response_times, 's--', label='Maximum', linewidth=2, markersize=8)
        axes[0, 0].set_xlabel('Concurrent Requests')
        axes[0, 0].set_ylabel('Response Time (seconds)')
        axes[0, 0].set_title('Response Time vs Concurrency Level')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
    else:
        axes[0, 0].text(0.5, 0.5, 'No performance data available', ha='center', va='center')
        axes[0, 0].set_title('Response Time vs Concurrency Level')
    
    # 2. Success rate by concurrency
    if performance_results:
        success_rates = [performance_results[c]['success_rate'] for c in concurrency_levels]
        
        bars = axes[0, 1].bar(concurrency_levels, success_rates, color='lightgreen', alpha=0.7)
        axes[0, 1].set_xlabel('Concurrent Requests')
        axes[0, 1].set_ylabel('Success Rate (%)')
        axes[0, 1].set_title('Success Rate by Concurrency Level')
        axes[0, 1].set_ylim(0, 105)
        
        # Add value labels on bars
        for bar, rate in zip(bars, success_rates):
            axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                           f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3)
    else:
        axes[0, 1].text(0.5, 0.5, 'No performance data available', ha='center', va='center')
        axes[0, 1].set_title('Success Rate by Concurrency Level')
    
    # 3. Throughput analysis
    if performance_results:
        throughputs = [performance_results[c]['throughput'] for c in concurrency_levels]
        
        axes[1, 0].plot(concurrency_levels, throughputs, 'o-', color='orange', linewidth=2, markersize=8)
        axes[1, 0].set_xlabel('Concurrent Requests')
        axes[1, 0].set_ylabel('Throughput (requests/second)')
        axes[1, 0].set_title('API Throughput vs Concurrency')
        axes[1, 0].grid(True, alpha=0.3)
    else:
        axes[1, 0].text(0.5, 0.5, 'No performance data available', ha='center', va='center')
        axes[1, 0].set_title('API Throughput vs Concurrency')
    
    # 4. Response time distribution
    metrics_df = performance_tracker.get_dataframe()
    if not metrics_df.empty:
        successful_metrics = metrics_df[metrics_df['success'] == True]
        if not successful_metrics.empty:
            axes[1, 1].hist(successful_metrics['response_time'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
            axes[1, 1].axvline(successful_metrics['response_time'].mean(), color='red', linestyle='--', 
                              label=f'Mean: {successful_metrics["response_time"].mean():.3f}s')
            axes[1, 1].set_xlabel('Response Time (seconds)')
            axes[1, 1].set_ylabel('Frequency')
            axes[1, 1].set_title('Response Time Distribution')
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
        else:
            axes[1, 1].text(0.5, 0.5, 'No successful requests', ha='center', va='center')
            axes[1, 1].set_title('Response Time Distribution')
    else:
        axes[1, 1].text(0.5, 0.5, 'No metrics data available', ha='center', va='center')
        axes[1, 1].set_title('Response Time Distribution')
    
    plt.tight_layout()
    plt.show()
    
    # Print performance summary
    print("📈 Performance Summary:")
    summary = performance_tracker.summary()
    for key, value in summary.items():
        if isinstance(value, float):
            print(f"   {key}: {value:.3f}")
        else:
            print(f"   {key}: {value}")

# Create visualizations
create_performance_visualizations(performance_results, performance_tracker)

## 5. Data Visualization

Visualizing test results, performance metrics, and search data.

In [ ]:
async def performance_test_concurrent_requests():
    """Test API performance with concurrent requests"""
    if not auth_success:
        print("❌ Authentication required for performance testing")
        return
    
    print("⚡ Performance Testing - Concurrent Requests\n")
    
    async def single_search_request(query_id: int) -> Dict[str, Any]:
        """Perform a single search request"""
        start_time = time.time()
        try:
            result = await api_client.search_by_skills(
                skills="Python,JavaScript",
                min_score=0.3
            )
            response_time = time.time() - start_time
            
            performance_tracker.record(
                "/api/v1/search/skills", "GET", response_time,
                result["status_code"], result["status_code"] == 200
            )
            
            return {
                "query_id": query_id,
                "success": result["status_code"] == 200,
                "response_time": response_time,
                "status_code": result["status_code"]
            }
        except Exception as e:
            response_time = time.time() - start_time
            return {
                "query_id": query_id,
                "success": False,
                "response_time": response_time,
                "error": str(e)
            }
    
    # Test different concurrency levels
    concurrency_levels = [1, 5, 10]
    performance_results = {}
    
    for concurrency in concurrency_levels:
        print(f"Testing with {concurrency} concurrent requests...")
        
        # Create tasks for concurrent execution
        tasks = [single_search_request(i) for i in range(concurrency)]
        
        # Measure total time for all concurrent requests
        start_time = time.time()
        results = await asyncio.gather(*tasks, return_exceptions=True)
        total_time = time.time() - start_time
        
        # Process results
        successful_requests = [r for r in results if isinstance(r, dict) and r.get("success", False)]
        failed_requests = [r for r in results if isinstance(r, dict) and not r.get("success", False)]
        
        if successful_requests:
            avg_response_time = sum(r["response_time"] for r in successful_requests) / len(successful_requests)
            max_response_time = max(r["response_time"] for r in successful_requests)
            min_response_time = min(r["response_time"] for r in successful_requests)
        else:
            avg_response_time = max_response_time = min_response_time = 0
        
        success_rate = len(successful_requests) / len(results) * 100
        throughput = len(results) / total_time
        
        performance_results[concurrency] = {
            "total_requests": len(results),
            "successful_requests": len(successful_requests),
            "failed_requests": len(failed_requests),
            "success_rate": success_rate,
            "total_time": total_time,
            "avg_response_time": avg_response_time,
            "min_response_time": min_response_time,
            "max_response_time": max_response_time,
            "throughput": throughput
        }
        
        print(f"   ✅ Success rate: {success_rate:.1f}%")
        print(f"   📊 Avg response time: {avg_response_time:.3f}s")
        print(f"   🚀 Throughput: {throughput:.1f} req/s")
        print(f"   ⏱️ Total time: {total_time:.3f}s\n")
    
    return performance_results

# Run performance tests
performance_results = await performance_test_concurrent_requests()

## 4. Performance Testing

Testing API performance with concurrent requests and measuring response times.

In [ ]:
async def test_search_functionality():
    """Test various search endpoints"""
    if not auth_success:
        print("❌ Authentication required for search testing")
        return
    
    print("🔍 Testing Search Functionality\n")
    
    search_results = {}
    
    # Test 1: Skills-based search
    print("1. Testing Skills-based Search...")
    start_time = time.time()
    
    skills_result = await api_client.search_by_skills(
        skills="Python,JavaScript,React",
        min_score=0.3
    )
    
    response_time = time.time() - start_time
    performance_tracker.record(
        "/api/v1/search/skills", "GET", response_time,
        skills_result["status_code"], skills_result["status_code"] == 200
    )
    
    print(f"   Status: {skills_result['status_code']}")
    print(f"   Response time: {response_time:.3f}s")
    
    if skills_result["status_code"] == 200:
        data = skills_result["data"]
        if isinstance(data, dict):
            total_results = data.get("total_results", 0)
            print(f"   ✅ Found {total_results} candidates")
            search_results["skills_search"] = data
        else:
            print("   ✅ Skills search completed")
    else:
        print(f"   ❌ Skills search failed: {skills_result['data']}")
    
    # Test 2: Advanced candidate search
    print("\n2. Testing Advanced Candidate Search...")
    search_params = {
        "query": "software engineer",
        "search_type": "skills_match",
        "skills": ["Python", "Django", "PostgreSQL"],
        "min_experience_years": 2,
        "limit": 20
    }
    
    start_time = time.time()
    candidate_result = await api_client.search_candidates(search_params)
    
    response_time = time.time() - start_time
    performance_tracker.record(
        "/api/v1/search/candidates", "POST", response_time,
        candidate_result["status_code"], candidate_result["status_code"] == 200
    )
    
    print(f"   Status: {candidate_result['status_code']}")
    print(f"   Response time: {response_time:.3f}s")
    
    if candidate_result["status_code"] == 200:
        data = candidate_result["data"]
        if isinstance(data, dict):
            total_results = data.get("total_results", 0)
            print(f"   ✅ Found {total_results} candidates")
            search_results["candidate_search"] = data
        else:
            print("   ✅ Candidate search completed")
    else:
        print(f"   ❌ Candidate search failed: {candidate_result['data']}")
    
    # Test 3: Smart search with AI
    print("\n3. Testing AI-Powered Smart Search...")
    smart_queries = [
        "Find senior Python developers with 5+ years experience",
        "Show me frontend developers who know React",
        "Search for data scientists with machine learning skills"
    ]
    
    for i, query in enumerate(smart_queries, 1):
        print(f"   Query {i}: {query}")
        start_time = time.time()
        
        smart_result = await api_client.smart_search(query, include_reasoning=True)
        
        response_time = time.time() - start_time
        performance_tracker.record(
            "/api/v1/search/smart", "POST", response_time,
            smart_result["status_code"], smart_result["status_code"] == 200
        )
        
        print(f"     Status: {smart_result['status_code']}")
        print(f"     Response time: {response_time:.3f}s")
        
        if smart_result["status_code"] == 200:
            data = smart_result["data"]
            if isinstance(data, dict):
                results_count = len(data.get("results", []))
                print(f"     ✅ Found {results_count} candidates")
                if "reasoning" in data:
                    print(f"     🤖 AI Reasoning: {data['reasoning'][:100]}...")
                search_results[f"smart_search_{i}"] = data
            else:
                print("     ✅ Smart search completed")
        else:
            print(f"     ❌ Smart search failed: {smart_result['data']}")
        
        if i < len(smart_queries):
            print()
    
    # Test 4: Get available search filters
    print("\n4. Testing Search Filters Endpoint...")
    start_time = time.time()
    
    filters_result = await api_client.get_search_filters()
    
    response_time = time.time() - start_time
    performance_tracker.record(
        "/api/v1/search/filters", "GET", response_time,
        filters_result["status_code"], filters_result["status_code"] == 200
    )
    
    print(f"   Status: {filters_result['status_code']}")
    print(f"   Response time: {response_time:.3f}s")
    
    if filters_result["status_code"] == 200:
        data = filters_result["data"]
        if isinstance(data, dict):
            companies_count = len(data.get("companies", []))
            departments_count = len(data.get("departments", []))
            skills_count = len(data.get("skills", []))
            print(f"   ✅ Available filters: {companies_count} companies, {departments_count} departments, {skills_count} skills")
            search_results["filters"] = data
        else:
            print("   ✅ Filters retrieved")
    else:
        print(f"   ❌ Filters retrieval failed: {filters_result['data']}")
    
    return search_results

# Run search tests
search_results = await test_search_functionality()

## 3. Search Functionality Testing

Comprehensive testing of all search endpoints including skills, departments, and smart search.

In [ ]:
def create_sample_pdf() -> bytes:
    """Create a simple PDF file for testing"""
    pdf_content = b"""%PDF-1.4
1 0 obj
<<
/Type /Catalog
/Pages 2 0 R
>>
endobj

2 0 obj
<<
/Type /Pages
/Kids [3 0 R]
/Count 1
>>
endobj

3 0 obj
<<
/Type /Page
/Parent 2 0 R
/MediaBox [0 0 612 792]
/Contents 4 0 R
>>
endobj

4 0 obj
<<
/Length 44
>>
stream
BT
/F1 12 Tf
100 700 Td
(John Doe - Software Engineer) Tj
ET
endstream
endobj

xref
0 5
0000000000 65535 f 
0000000009 00000 n 
0000000074 00000 n 
0000000120 00000 n 
0000000179 00000 n 
trailer
<<
/Size 5
/Root 1 0 R
>>
startxref
238
%%EOF"""
    return pdf_content


async def test_resume_upload():
    """Test resume upload functionality"""
    if not auth_success:
        print("❌ Authentication required for upload testing")
        return
    
    print("📄 Testing Resume Upload\n")
    
    # Test PDF upload
    print("1. Testing PDF Upload...")
    pdf_content = create_sample_pdf()
    
    start_time = time.time()
    upload_result = await api_client.upload_resume(
        file_content=pdf_content,
        filename="john_doe_resume.pdf",
        content_type="application/pdf"
    )
    
    response_time = time.time() - start_time
    performance_tracker.record(
        "/api/v1/resumes/upload", "POST", response_time,
        upload_result["status_code"], upload_result["status_code"] in [200, 201]
    )
    
    print(f"   Status: {upload_result['status_code']}")
    print(f"   Response time: {response_time:.3f}s")
    
    if upload_result["status_code"] in [200, 201]:
        print("   ✅ PDF upload successful")
        if isinstance(upload_result["data"], dict):
            resume_id = upload_result["data"].get("resume_id")
            if resume_id:
                print(f"   📋 Resume ID: {resume_id}")
            parsing_status = upload_result["data"].get("parsing_status", "unknown")
            print(f"   🔄 Parsing status: {parsing_status}")
    else:
        print(f"   ❌ Upload failed: {upload_result['data']}")
    
    # Test invalid file upload
    print("\n2. Testing Invalid File Upload...")
    invalid_content = b"This is not a valid resume file"
    
    start_time = time.time()
    invalid_upload = await api_client.upload_resume(
        file_content=invalid_content,
        filename="invalid.txt",
        content_type="text/plain"
    )
    
    response_time = time.time() - start_time
    performance_tracker.record(
        "/api/v1/resumes/upload", "POST", response_time,
        invalid_upload["status_code"], invalid_upload["status_code"] >= 400
    )
    
    print(f"   Status: {invalid_upload['status_code']}")
    print(f"   Response time: {response_time:.3f}s")
    
    if invalid_upload["status_code"] >= 400:
        print("   ✅ Invalid file correctly rejected")
    else:
        print("   ⚠️ Invalid file was accepted (unexpected)")

# Run upload tests
await test_resume_upload()

## 2. Resume Upload Testing

Testing file upload functionality with sample PDF files.

In [ ]:
async def test_authentication():
    """Test authentication flow"""
    print("🔐 Testing Authentication Flow\n")
    
    # Test user registration
    print("1. Testing User Registration...")
    start_time = time.time()
    
    register_result = await api_client.register(
        email="demo.user@example.com",
        username="demouser",
        password="DemoPassword123!",
        full_name="Demo User"
    )
    
    response_time = time.time() - start_time
    performance_tracker.record(
        "/api/v1/auth/register", "POST", response_time, 
        register_result["status_code"], register_result["status_code"] in [200, 201]
    )
    
    print(f"   Status: {register_result['status_code']}")
    print(f"   Response time: {response_time:.3f}s")
    if register_result["status_code"] in [200, 201]:
        print("   ✅ Registration successful")
    else:
        print(f"   ❌ Registration failed: {register_result['data']}")
    
    # Test user login
    print("\n2. Testing User Login...")
    start_time = time.time()
    
    login_result = await api_client.login(
        email="demo.user@example.com",
        password="DemoPassword123!"
    )
    
    response_time = time.time() - start_time
    performance_tracker.record(
        "/api/v1/auth/login", "POST", response_time,
        login_result["status_code"], login_result["status_code"] == 200
    )
    
    print(f"   Status: {login_result['status_code']}")
    print(f"   Response time: {response_time:.3f}s")
    if login_result["status_code"] == 200:
        print("   ✅ Login successful")
        print(f"   🔑 Token acquired: {api_client.token[:20]}...")
        return True
    else:
        print(f"   ❌ Login failed: {login_result['data']}")
        return False

# Run authentication test
auth_success = await test_authentication()

## 1. Authentication Flow Testing

Testing user registration and login functionality.

In [ ]:
class APIClient:
    """Helper class for API interactions"""
    
    def __init__(self, base_url: str = "http://localhost:8000"):
        self.base_url = base_url
        self.client = httpx.AsyncClient(timeout=30.0)
        self.token = None
        self.headers = {"Content-Type": "application/json"}
    
    async def register(self, email: str, username: str, password: str, full_name: str) -> Dict[str, Any]:
        """Register a new user"""
        data = {
            "email": email,
            "username": username,
            "password": password,
            "full_name": full_name
        }
        response = await self.client.post(f"{self.base_url}/api/v1/auth/register", json=data)
        return {"status_code": response.status_code, "data": response.json() if response.status_code < 400 else response.text}
    
    async def login(self, email: str, password: str) -> Dict[str, Any]:
        """Login and store authentication token"""
        data = {"username": email, "password": password}
        response = await self.client.post(
            f"{self.base_url}/api/v1/auth/login",
            data=data,
            headers={"Content-Type": "application/x-www-form-urlencoded"}
        )
        
        if response.status_code == 200:
            result = response.json()
            self.token = result.get("access_token")
            self.headers["Authorization"] = f"Bearer {self.token}"
            return {"status_code": response.status_code, "data": result}
        else:
            return {"status_code": response.status_code, "data": response.text}
    
    async def upload_resume(self, file_content: bytes, filename: str, content_type: str) -> Dict[str, Any]:
        """Upload a resume file"""
        files = {"file": (filename, file_content, content_type)}
        headers = {"Authorization": self.headers.get("Authorization", "")}
        
        response = await self.client.post(
            f"{self.base_url}/api/v1/resumes/upload",
            files=files,
            headers=headers
        )
        return {"status_code": response.status_code, "data": response.json() if response.status_code < 400 else response.text}
    
    async def search_candidates(self, query_params: Dict[str, Any]) -> Dict[str, Any]:
        """Search candidates with various parameters"""
        response = await self.client.post(
            f"{self.base_url}/api/v1/search/candidates",
            json=query_params,
            headers=self.headers
        )
        return {"status_code": response.status_code, "data": response.json() if response.status_code < 400 else response.text}
    
    async def search_by_skills(self, skills: str, min_score: float = 0.3) -> Dict[str, Any]:
        """Search candidates by skills"""
        params = {"skills": skills, "min_score": min_score}
        response = await self.client.get(
            f"{self.base_url}/api/v1/search/skills",
            params=params,
            headers=self.headers
        )
        return {"status_code": response.status_code, "data": response.json() if response.status_code < 400 else response.text}
    
    async def smart_search(self, query: str, include_reasoning: bool = True) -> Dict[str, Any]:
        """Perform AI-powered smart search"""
        data = {"query": query, "include_reasoning": include_reasoning}
        response = await self.client.post(
            f"{self.base_url}/api/v1/search/smart",
            json=data,
            headers=self.headers
        )
        return {"status_code": response.status_code, "data": response.json() if response.status_code < 400 else response.text}
    
    async def get_search_filters(self) -> Dict[str, Any]:
        """Get available search filters"""
        response = await self.client.get(
            f"{self.base_url}/api/v1/search/filters",
            headers=self.headers
        )
        return {"status_code": response.status_code, "data": response.json() if response.status_code < 400 else response.text}
    
    async def close(self):
        """Close the HTTP client"""
        await self.client.aclose()


class PerformanceTracker:
    """Track API performance metrics"""
    
    def __init__(self):
        self.metrics = []
    
    def record(self, endpoint: str, method: str, response_time: float, status_code: int, success: bool):
        """Record a performance metric"""
        self.metrics.append({
            "endpoint": endpoint,
            "method": method,
            "response_time": response_time,
            "status_code": status_code,
            "success": success,
            "timestamp": datetime.now()
        })
    
    def get_dataframe(self) -> pd.DataFrame:
        """Get metrics as pandas DataFrame"""
        return pd.DataFrame(self.metrics)
    
    def summary(self) -> Dict[str, Any]:
        """Get performance summary"""
        if not self.metrics:
            return {"message": "No metrics recorded"}
        
        df = self.get_dataframe()
        return {
            "total_requests": len(df),
            "success_rate": df['success'].mean() * 100,
            "avg_response_time": df['response_time'].mean(),
            "min_response_time": df['response_time'].min(),
            "max_response_time": df['response_time'].max(),
            "p95_response_time": df['response_time'].quantile(0.95),
            "endpoints_tested": df['endpoint'].nunique()
        }

# Initialize clients
api_client = APIClient()
performance_tracker = PerformanceTracker()

print("✅ Helper classes initialized")

## Helper Classes and Functions

These helper classes provide convenient methods for API interaction and performance tracking.

In [ ]:
# Import required libraries
import httpx
import asyncio
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import Dict, List, Any, Optional
import io
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

# API Builder - Comprehensive API Testing Notebook

This notebook provides comprehensive testing and demonstration of the HR Resume Search API endpoints.

## Features Covered:
- Authentication flow testing
- Resume upload testing with sample files
- Search functionality demonstrations
- Performance testing with concurrent requests
- Data visualization of search results

## Requirements:
- API server running on localhost:8000
- Python packages: httpx, matplotlib, seaborn, pandas, asyncio

# HR Resume Search MCP API - Comprehensive Testing Suite

This notebook provides comprehensive testing and demonstration of the HR Resume Search MCP API.

## Features Tested
- ✅ **Authentication Flow**: JWT login, token management, refresh
- ✅ **Resume Upload**: PDF/DOC/DOCX file processing with Claude AI
- ✅ **Search Functionality**: Smart candidate matching and filtering
- ✅ **Performance Testing**: Concurrent requests and load testing
- ✅ **Data Visualization**: Search results and performance metrics

## Prerequisites
1. API server running at `http://localhost:8000`
2. Database and Redis services available
3. Claude API key configured

---

## 📦 Setup and Configuration

In [ ]:
# Import required libraries
import os
import sys
import json
import asyncio
import httpx
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
import time
from pathlib import Path
import uuid
import base64
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# IPython display
from IPython.display import display, HTML, JSON, Markdown
import ipywidgets as widgets
from ipywidgets import interact, fixed, IntSlider

# Load environment variables
from dotenv import load_dotenv
load_dotenv('../.env')

# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))

# Configure matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configure seaborn
sns.set_palette("husl")

print("✅ All libraries imported successfully")

In [ ]:
# Configuration
API_HOST = os.getenv('API_HOST', 'localhost')
API_PORT = os.getenv('API_PORT', '8000')
API_PREFIX = os.getenv('API_PREFIX', '/api/v1')
API_BASE_URL = f"http://{API_HOST}:{API_PORT}"
API_URL = f"{API_BASE_URL}{API_PREFIX}"

# Test configuration
TEST_TIMEOUT = 30
CONCURRENT_REQUESTS = 10
LOAD_TEST_DURATION = 60  # seconds

# Display configuration
config_html = f"""
<div style="background-color: #f0f8ff; padding: 15px; border-radius: 10px; border-left: 5px solid #007acc;">
    <h3 style="color: #007acc; margin-top: 0;">🔧 API Configuration</h3>
    <table style="border-collapse: collapse; width: 100%;">
        <tr><td><strong>Base URL:</strong></td><td>{API_BASE_URL}</td></tr>
        <tr><td><strong>API URL:</strong></td><td>{API_URL}</td></tr>
        <tr><td><strong>Timeout:</strong></td><td>{TEST_TIMEOUT}s</td></tr>
        <tr><td><strong>Environment:</strong></td><td>{os.getenv('ENVIRONMENT', 'development')}</td></tr>
    </table>
</div>
"""

display(HTML(config_html))
print(f"API Configuration loaded successfully")

## 🛠️ Helper Functions and Test Utilities

In [ ]:
class APITestClient:
    """Enhanced API testing client with authentication and metrics"""
    
    def __init__(self, base_url: str = API_URL):
        self.base_url = base_url
        self.client = httpx.Client(timeout=TEST_TIMEOUT)
        self.async_client = None
        self.auth_token = None
        self.headers = {"Content-Type": "application/json"}
        self.metrics = {
            "requests_made": 0,
            "requests_successful": 0,
            "requests_failed": 0,
            "total_response_time": 0,
            "errors": []
        }
    
    def set_auth_token(self, token: str):
        """Set authentication token"""
        self.auth_token = token
        self.headers["Authorization"] = f"Bearer {token}"
    
    def _make_request(self, method: str, endpoint: str, **kwargs) -> Dict[str, Any]:
        """Make HTTP request with metrics tracking"""
        url = f"{self.base_url}{endpoint}" if not endpoint.startswith('http') else endpoint
        start_time = time.time()
        
        try:
            response = getattr(self.client, method.lower())(
                url, headers=self.headers, **kwargs
            )
            response_time = time.time() - start_time
            
            # Update metrics
            self.metrics["requests_made"] += 1
            self.metrics["total_response_time"] += response_time
            
            if response.status_code < 400:
                self.metrics["requests_successful"] += 1
            else:
                self.metrics["requests_failed"] += 1
                self.metrics["errors"].append({
                    "endpoint": endpoint,
                    "status_code": response.status_code,
                    "error": response.text
                })
            
            return {
                "success": response.status_code < 400,
                "status_code": response.status_code,
                "response_time": response_time,
                "data": response.json() if response.content else None,
                "headers": dict(response.headers),
                "url": url
            }
            
        except Exception as e:
            response_time = time.time() - start_time
            self.metrics["requests_made"] += 1
            self.metrics["requests_failed"] += 1
            self.metrics["total_response_time"] += response_time
            self.metrics["errors"].append({
                "endpoint": endpoint,
                "error": str(e)
            })
            
            return {
                "success": False,
                "status_code": None,
                "response_time": response_time,
                "data": None,
                "error": str(e),
                "url": url
            }
    
    def get(self, endpoint: str, **kwargs) -> Dict[str, Any]:
        return self._make_request("GET", endpoint, **kwargs)
    
    def post(self, endpoint: str, **kwargs) -> Dict[str, Any]:
        return self._make_request("POST", endpoint, **kwargs)
    
    def put(self, endpoint: str, **kwargs) -> Dict[str, Any]:
        return self._make_request("PUT", endpoint, **kwargs)
    
    def delete(self, endpoint: str, **kwargs) -> Dict[str, Any]:
        return self._make_request("DELETE", endpoint, **kwargs)
    
    async def async_get(self, endpoint: str, **kwargs) -> Dict[str, Any]:
        """Async GET request"""
        if not self.async_client:
            self.async_client = httpx.AsyncClient(timeout=TEST_TIMEOUT)
        
        url = f"{self.base_url}{endpoint}" if not endpoint.startswith('http') else endpoint
        start_time = time.time()
        
        try:
            response = await self.async_client.get(
                url, headers=self.headers, **kwargs
            )
            response_time = time.time() - start_time
            
            return {
                "success": response.status_code < 400,
                "status_code": response.status_code,
                "response_time": response_time,
                "data": response.json() if response.content else None,
                "url": url
            }
        except Exception as e:
            return {
                "success": False,
                "status_code": None,
                "response_time": time.time() - start_time,
                "error": str(e),
                "url": url
            }
    
    def get_metrics(self) -> Dict[str, Any]:
        """Get performance metrics"""
        metrics = self.metrics.copy()
        if metrics["requests_made"] > 0:
            metrics["average_response_time"] = metrics["total_response_time"] / metrics["requests_made"]
            metrics["success_rate"] = (metrics["requests_successful"] / metrics["requests_made"]) * 100
        else:
            metrics["average_response_time"] = 0
            metrics["success_rate"] = 0
        return metrics
    
    def reset_metrics(self):
        """Reset performance metrics"""
        self.metrics = {
            "requests_made": 0,
            "requests_successful": 0,
            "requests_failed": 0,
            "total_response_time": 0,
            "errors": []
        }
    
    def close(self):
        """Close client connections"""
        self.client.close()
        if self.async_client:
            asyncio.run(self.async_client.aclose())

# Initialize test client
api_client = APITestClient()

print("✅ APITestClient initialized successfully")

In [ ]:
def create_sample_resume_files():
    """Create sample resume files for testing"""
    sample_resumes = {
        "john_doe_resume.json": {
            "name": "John Doe",
            "email": "john.doe@example.com",
            "phone": "+1-555-0123",
            "location": "New York, NY",
            "summary": "Experienced software engineer with 5+ years in full-stack development",
            "experience": [
                {
                    "company": "Tech Corp",
                    "position": "Senior Software Engineer",
                    "department": "Engineering",
                    "desk": "Platform Team",
                    "start_date": "2020-01-01",
                    "end_date": None,
                    "description": "Leading platform development initiatives"
                },
                {
                    "company": "StartupXYZ",
                    "position": "Software Engineer",
                    "department": "Product",
                    "desk": "Backend Team",
                    "start_date": "2018-06-01",
                    "end_date": "2019-12-31",
                    "description": "Developed scalable backend services"
                }
            ],
            "skills": ["Python", "JavaScript", "Docker", "AWS", "PostgreSQL"],
            "education": [
                {
                    "institution": "University of Technology",
                    "degree": "Bachelor of Science",
                    "field_of_study": "Computer Science",
                    "graduation_date": "2018-05-01"
                }
            ]
        },
        "jane_smith_resume.json": {
            "name": "Jane Smith",
            "email": "jane.smith@example.com",
            "phone": "+1-555-0456",
            "location": "San Francisco, CA",
            "summary": "Product manager with expertise in data analytics and machine learning",
            "experience": [
                {
                    "company": "Tech Corp",
                    "position": "Senior Product Manager",
                    "department": "Product",
                    "desk": "AI Team",
                    "start_date": "2019-03-01",
                    "end_date": None,
                    "description": "Leading AI product initiatives"
                },
                {
                    "company": "DataCorp",
                    "position": "Data Analyst",
                    "department": "Analytics",
                    "desk": "Business Intelligence",
                    "start_date": "2017-01-01",
                    "end_date": "2019-02-28",
                    "description": "Analyzed business metrics and created dashboards"
                }
            ],
            "skills": ["Python", "SQL", "Tableau", "Machine Learning", "Product Strategy"],
            "education": [
                {
                    "institution": "Stanford University",
                    "degree": "Master of Science",
                    "field_of_study": "Data Science",
                    "graduation_date": "2016-06-01"
                }
            ]
        }
    }
    
    # Create sample files directory
    sample_dir = Path("sample_resumes")
    sample_dir.mkdir(exist_ok=True)
    
    # Save sample files
    for filename, data in sample_resumes.items():
        with open(sample_dir / filename, 'w') as f:
            json.dump(data, f, indent=2)
    
    return sample_dir, list(sample_resumes.keys())

def visualize_test_results(results: List[Dict], title: str = "Test Results"):
    """Visualize test results with multiple charts"""
    df = pd.DataFrame(results)
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Response Times', 'Success Rate', 'Status Codes', 'Timeline'),
        specs=[[{"secondary_y": False}, {"type": "pie"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )
    
    # Response times histogram
    fig.add_trace(
        go.Histogram(x=df['response_time'], name="Response Time", nbinsx=20),
        row=1, col=1
    )
    
    # Success rate pie chart
    success_counts = df['success'].value_counts()
    fig.add_trace(
        go.Pie(labels=['Success', 'Failed'], values=[success_counts.get(True, 0), success_counts.get(False, 0)]),
        row=1, col=2
    )
    
    # Status codes bar chart
    status_counts = df['status_code'].value_counts()
    fig.add_trace(
        go.Bar(x=status_counts.index.astype(str), y=status_counts.values, name="Status Codes"),
        row=2, col=1
    )
    
    # Timeline scatter plot
    if 'timestamp' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['response_time'], mode='lines+markers', name="Response Time Timeline"),
            row=2, col=2
        )
    
    fig.update_layout(height=700, title_text=title, showlegend=False)
    fig.show()
    
    # Display summary statistics
    stats_html = f"""
    <div style="background-color: #f8f9fa; padding: 15px; border-radius: 10px; margin: 10px 0;">
        <h4>📊 Test Summary Statistics</h4>
        <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 15px;">
            <div style="text-align: center; padding: 10px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #007acc;">{len(df)}</div>
                <div>Total Requests</div>
            </div>
            <div style="text-align: center; padding: 10px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #28a745;">{df['success'].sum()}</div>
                <div>Successful</div>
            </div>
            <div style="text-align: center; padding: 10px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #dc3545;">{(~df['success']).sum()}</div>
                <div>Failed</div>
            </div>
            <div style="text-align: center; padding: 10px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #6f42c1;">{df['response_time'].mean():.3f}s</div>
                <div>Avg Response Time</div>
            </div>
        </div>
    </div>
    """
    display(HTML(stats_html))

# Create sample data
sample_dir, sample_files = create_sample_resume_files()
print(f"✅ Created sample resume files: {sample_files}")
print(f"✅ Helper functions initialized successfully")

## 🔍 API Health Check

First, let's verify that the API is running and accessible.

In [ ]:
# Test basic API connectivity
print("🔍 Testing API connectivity...")

health_endpoints = [
    ("/", "Root endpoint"),
    ("/health", "Health check"),
    ("/readiness", "Readiness check"),
    ("/docs", "API documentation"),
    ("/openapi.json", "OpenAPI schema")
]

health_results = []

for endpoint, description in health_endpoints:
    # Use base URL for root-level endpoints
    test_url = f"{API_BASE_URL}{endpoint}" if endpoint in ["/", "/health", "/readiness", "/docs", "/openapi.json"] else endpoint
    result = api_client.get(test_url)
    
    health_results.append({
        "endpoint": endpoint,
        "description": description,
        "status_code": result["status_code"],
        "success": result["success"],
        "response_time": result["response_time"],
        "timestamp": datetime.now()
    })
    
    status_icon = "✅" if result["success"] else "❌"
    print(f"{status_icon} {endpoint} ({description}): {result['status_code']} - {result['response_time']:.3f}s")

# Visualize health check results
visualize_test_results(health_results, "API Health Check Results")

# Check if API is ready
api_ready = all(result["success"] for result in health_results if result["endpoint"] in ["/", "/health"])
if api_ready:
    print("\n🎉 API is ready for testing!")
else:
    print("\n⚠️ API may not be fully ready. Some tests may fail.")

---

## 🔐 Authentication Flow Testing

Testing JWT authentication, login, token refresh, and logout flows.

In [ ]:
# Authentication test data
test_users = {
    "admin": {
        "email": "admin@example.com",
        "password": "admin123",
        "role": "admin"
    },
    "hr_manager": {
        "email": "hr@example.com",
        "password": "hr123",
        "role": "hr_manager"
    },
    "recruiter": {
        "email": "recruiter@example.com",
        "password": "recruiter123",
        "role": "recruiter"
    }
}

print("🔐 Testing Authentication Flow...")
auth_results = []

# Test user registration (if endpoint exists)
print("\n1. Testing User Registration")
for username, user_data in test_users.items():
    register_data = {
        "email": user_data["email"],
        "password": user_data["password"],
        "role": user_data["role"]
    }
    
    result = api_client.post("/auth/register", json=register_data)
    auth_results.append({
        "test": f"Register {username}",
        "success": result["success"],
        "status_code": result["status_code"],
        "response_time": result["response_time"],
        "timestamp": datetime.now()
    })
    
    status_icon = "✅" if result["success"] or result["status_code"] == 409 else "❌"  # 409 = user exists
    print(f"{status_icon} Register {username}: {result['status_code']}")

# Test login
print("\n2. Testing User Login")
successful_logins = {}

for username, user_data in test_users.items():
    login_data = {
        "email": user_data["email"],
        "password": user_data["password"]
    }
    
    result = api_client.post("/auth/login", json=login_data)
    auth_results.append({
        "test": f"Login {username}",
        "success": result["success"],
        "status_code": result["status_code"],
        "response_time": result["response_time"],
        "timestamp": datetime.now()
    })
    
    if result["success"] and result["data"]:
        token = result["data"].get("access_token")
        if token:
            successful_logins[username] = {
                "token": token,
                "user_data": result["data"]
            }
            print(f"✅ Login {username}: Success - Token obtained")
        else:
            print(f"❌ Login {username}: No token in response")
    else:
        print(f"❌ Login {username}: {result.get('status_code', 'Unknown error')}")

# Test authenticated endpoints
print("\n3. Testing Authenticated Endpoints")
if successful_logins:
    # Use first successful login for testing
    test_user = list(successful_logins.keys())[0]
    test_token = successful_logins[test_user]["token"]
    
    # Set token for API client
    api_client.set_auth_token(test_token)
    
    # Test protected endpoints
    protected_endpoints = [
        "/auth/me",
        "/resumes",
        "/search"
    ]
    
    for endpoint in protected_endpoints:
        result = api_client.get(endpoint)
        auth_results.append({
            "test": f"Protected {endpoint}",
            "success": result["success"],
            "status_code": result["status_code"],
            "response_time": result["response_time"],
            "timestamp": datetime.now()
        })
        
        status_icon = "✅" if result["success"] else "❌"
        print(f"{status_icon} {endpoint}: {result['status_code']}")
    
    print(f"\n🔑 Using token for {test_user}: {test_token[:20]}...")
else:
    print("❌ No successful logins - cannot test protected endpoints")

# Display authentication test results
print("\n📊 Authentication Test Summary:")
visualize_test_results(auth_results, "Authentication Flow Test Results")

---

## 📄 Resume Upload Testing

Testing file upload functionality with various formats and Claude AI parsing.

In [ ]:
print("📄 Testing Resume Upload Functionality...")
upload_results = []

# Create test files in different formats
def create_test_resume_content():
    return """
JOHN DOE
Software Engineer
Email: john.doe@example.com
Phone: +1-555-0123
Location: New York, NY

SUMMARY
Experienced software engineer with 5+ years in full-stack development.
Expertise in Python, JavaScript, and cloud technologies.

EXPERIENCE
Senior Software Engineer | Tech Corp | 2020-Present
- Lead platform development initiatives
- Manage team of 5 developers
- Implemented microservices architecture

Software Engineer | StartupXYZ | 2018-2019
- Developed scalable backend services
- Built REST APIs and databases
- Worked with Docker and Kubernetes

SKILLS
Programming: Python, JavaScript, Go, SQL
Technologies: Docker, Kubernetes, AWS, PostgreSQL
Frameworks: FastAPI, React, Node.js

EDUCATION
Bachelor of Science in Computer Science
University of Technology | 2018
"""

# Create test files
test_files_dir = Path("test_uploads")
test_files_dir.mkdir(exist_ok=True)

# Create text file (simulating converted PDF content)
test_content = create_test_resume_content()
test_files = []

# Text file
text_file = test_files_dir / "john_doe_resume.txt"
with open(text_file, 'w') as f:
    f.write(test_content)
test_files.append(("john_doe_resume.txt", "text/plain"))

# JSON file (structured resume)
json_file = test_files_dir / "jane_smith_resume.json"
with open(json_file, 'w') as f:
    json.dump({
        "name": "Jane Smith",
        "email": "jane.smith@example.com",
        "position": "Product Manager",
        "experience": [
            {
                "company": "Tech Corp",
                "role": "Senior Product Manager",
                "department": "Product",
                "years": 3
            }
        ],
        "skills": ["Product Strategy", "Data Analysis", "Python", "SQL"]
    }, f, indent=2)
test_files.append(("jane_smith_resume.json", "application/json"))

print(f"Created test files: {[f[0] for f in test_files]}")

# Test file uploads
print("\n1. Testing File Upload Endpoints")
uploaded_resumes = []

for filename, content_type in test_files:
    file_path = test_files_dir / filename
    
    # Test upload
    try:
        with open(file_path, 'rb') as f:
            files = {'file': (filename, f, content_type)}
            # Note: files parameter bypasses the json content-type header
            result = api_client.client.post(
                f"{api_client.base_url}/resumes/upload",
                files=files,
                headers={"Authorization": api_client.headers.get("Authorization", "")}
            )
            
            upload_result = {
                "success": result.status_code < 400,
                "status_code": result.status_code,
                "response_time": 0,  # Will be updated by client metrics
                "data": result.json() if result.content else None,
                "filename": filename
            }
            
            upload_results.append({
                "test": f"Upload {filename}",
                "success": upload_result["success"],
                "status_code": upload_result["status_code"],
                "response_time": upload_result["response_time"],
                "timestamp": datetime.now(),
                "file_type": content_type
            })
            
            if upload_result["success"] and upload_result["data"]:
                resume_id = upload_result["data"].get("id")
                if resume_id:
                    uploaded_resumes.append({
                        "id": resume_id,
                        "filename": filename,
                        "data": upload_result["data"]
                    })
                    print(f"✅ Uploaded {filename}: ID {resume_id}")
                else:
                    print(f"✅ Uploaded {filename}: No ID returned")
            else:
                print(f"❌ Upload {filename}: {upload_result['status_code']}")
                
    except Exception as e:
        print(f"❌ Upload {filename}: Error - {str(e)}")
        upload_results.append({
            "test": f"Upload {filename}",
            "success": False,
            "status_code": None,
            "response_time": 0,
            "timestamp": datetime.now(),
            "error": str(e)
        })

# Test resume parsing status
print("\n2. Testing Resume Parsing Status")
for resume in uploaded_resumes:
    result = api_client.get(f"/resumes/{resume['id']}")
    upload_results.append({
        "test": f"Get resume {resume['id']}",
        "success": result["success"],
        "status_code": result["status_code"],
        "response_time": result["response_time"],
        "timestamp": datetime.now()
    })
    
    if result["success"] and result["data"]:
        parse_status = result["data"].get("parse_status", "unknown")
        print(f"✅ Resume {resume['id']}: Parse status = {parse_status}")
    else:
        print(f"❌ Resume {resume['id']}: Could not get status")

# Test Claude AI parsing (if available)
print("\n3. Testing Claude AI Parsing")
for resume in uploaded_resumes:
    result = api_client.get(f"/resumes/{resume['id']}/parse")
    upload_results.append({
        "test": f"Parse resume {resume['id']}",
        "success": result["success"],
        "status_code": result["status_code"],
        "response_time": result["response_time"],
        "timestamp": datetime.now()
    })
    
    status_icon = "✅" if result["success"] else "❌"
    print(f"{status_icon} Parse resume {resume['id']}: {result['status_code']}")
    
    if result["success"] and result["data"]:
        parsed_data = result["data"]
        print(f"   Parsed fields: {list(parsed_data.keys())}")

# Display upload test results
print("\n📊 Resume Upload Test Summary:")
visualize_test_results(upload_results, "Resume Upload Test Results")

# Store uploaded resume IDs for search testing
resume_ids = [r["id"] for r in uploaded_resumes]
print(f"\n📝 Uploaded resume IDs for search testing: {resume_ids}")

---

## 🔍 Search Functionality Testing

Testing various search capabilities including smart matching and filtering.

In [ ]:
print("🔍 Testing Search Functionality...")
search_results = []
search_data = []

# Define test search queries
search_queries = [
    {
        "name": "Basic keyword search",
        "endpoint": "/search",
        "params": {"q": "software engineer"},
        "description": "Search for software engineers"
    },
    {
        "name": "Skills-based search",
        "endpoint": "/search",
        "params": {"skills": "Python,JavaScript"},
        "description": "Search by specific skills"
    },
    {
        "name": "Department search",
        "endpoint": "/search",
        "params": {"department": "Engineering"},
        "description": "Search by department"
    },
    {
        "name": "Experience range",
        "endpoint": "/search",
        "params": {"min_experience": 3, "max_experience": 8},
        "description": "Search by experience range"
    },
    {
        "name": "Location search",
        "endpoint": "/search",
        "params": {"location": "New York"},
        "description": "Search by location"
    },
    {
        "name": "Advanced search",
        "endpoint": "/search/advanced",
        "method": "POST",
        "data": {
            "query": "senior engineer",
            "filters": {
                "skills": ["Python", "AWS"],
                "experience_years": {"min": 3, "max": 10},
                "departments": ["Engineering", "Product"]
            },
            "sort_by": "relevance",
            "limit": 20
        },
        "description": "Advanced search with multiple filters"
    }
]

# Test similar candidates search (if we have uploaded resumes)
if resume_ids:
    for resume_id in resume_ids[:2]:  # Test first 2 resumes
        search_queries.append({
            "name": f"Similar candidates to {resume_id}",
            "endpoint": f"/search/similar/{resume_id}",
            "params": {},
            "description": f"Find candidates similar to resume {resume_id}"
        })

# Execute search tests
print("\n1. Testing Search Endpoints")
for query in search_queries:
    try:
        if query.get("method") == "POST":
            result = api_client.post(query["endpoint"], json=query.get("data", {}))
        else:
            result = api_client.get(query["endpoint"], params=query.get("params", {}))
        
        search_results.append({
            "test": query["name"],
            "success": result["success"],
            "status_code": result["status_code"],
            "response_time": result["response_time"],
            "timestamp": datetime.now(),
            "endpoint": query["endpoint"]
        })
        
        status_icon = "✅" if result["success"] else "❌"
        print(f"{status_icon} {query['name']}: {result['status_code']} - {result['response_time']:.3f}s")
        
        # Store search data for visualization
        if result["success"] and result["data"]:
            results_data = result["data"].get("results", [])
            total_count = result["data"].get("total", 0)
            
            search_data.append({
                "query_name": query["name"],
                "total_results": total_count,
                "returned_results": len(results_data),
                "response_time": result["response_time"],
                "results": results_data[:5]  # Store first 5 results for analysis
            })
            
            print(f"   Found {total_count} total results, returned {len(results_data)}")
        
    except Exception as e:
        print(f"❌ {query['name']}: Error - {str(e)}")
        search_results.append({
            "test": query["name"],
            "success": False,
            "status_code": None,
            "response_time": 0,
            "timestamp": datetime.now(),
            "error": str(e)
        })

# Test professional network search
print("\n2. Testing Professional Network Search")
if resume_ids:
    for resume_id in resume_ids[:1]:  # Test first resume
        result = api_client.get(f"/search/network/{resume_id}")
        search_results.append({
            "test": f"Network search for {resume_id}",
            "success": result["success"],
            "status_code": result["status_code"],
            "response_time": result["response_time"],
            "timestamp": datetime.now()
        })
        
        status_icon = "✅" if result["success"] else "❌"
        print(f"{status_icon} Network search for {resume_id}: {result['status_code']}")

# Test search filters and pagination
print("\n3. Testing Search Filters and Pagination")
pagination_tests = [
    {"limit": 5, "offset": 0},
    {"limit": 10, "offset": 5},
    {"limit": 20, "offset": 0}
]

for params in pagination_tests:
    result = api_client.get("/search", params={"q": "engineer", **params})
    search_results.append({
        "test": f"Pagination limit={params['limit']} offset={params['offset']}",
        "success": result["success"],
        "status_code": result["status_code"],
        "response_time": result["response_time"],
        "timestamp": datetime.now()
    })
    
    status_icon = "✅" if result["success"] else "❌"
    print(f"{status_icon} Pagination test: {result['status_code']}")

# Visualize search test results
print("\n📊 Search Test Summary:")
visualize_test_results(search_results, "Search Functionality Test Results")

# Create search results visualization
if search_data:
    search_df = pd.DataFrame(search_data)
    
    # Search results chart
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Results count by query
    ax1.bar(range(len(search_df)), search_df['total_results'])
    ax1.set_title('Search Results Count by Query')
    ax1.set_xlabel('Query Index')
    ax1.set_ylabel('Total Results')
    ax1.tick_params(axis='x', rotation=45)
    
    # Response time by query
    ax2.plot(search_df['response_time'], marker='o', linewidth=2, markersize=6)
    ax2.set_title('Search Response Times')
    ax2.set_xlabel('Query Index')
    ax2.set_ylabel('Response Time (seconds)')
    ax2.grid(True, alpha=0.3)
    
    # Results distribution
    ax3.hist(search_df['total_results'], bins=10, edgecolor='black', alpha=0.7)
    ax3.set_title('Distribution of Search Result Counts')
    ax3.set_xlabel('Number of Results')
    ax3.set_ylabel('Frequency')
    
    # Performance vs results
    ax4.scatter(search_df['total_results'], search_df['response_time'], 
                s=100, alpha=0.6, c=range(len(search_df)), cmap='viridis')
    ax4.set_title('Response Time vs Results Count')
    ax4.set_xlabel('Total Results')
    ax4.set_ylabel('Response Time (seconds)')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📈 Analyzed {len(search_data)} search queries with results")
else:
    print("\n⚠️ No search data available for visualization")

print(f"\n🔍 Search testing completed - {len(search_results)} tests executed")

In [ ]:
print("🔬 Testing JSONB Query Capabilities...")

# Define JSONB query examples that demonstrate the power of PostgreSQL JSONB
jsonb_query_examples = [
    {
        "name": "Skills Array Search",
        "description": "Find candidates with specific skills using JSONB array operations",
        "query_type": "skills_search",
        "example_sql": """
        SELECT * FROM candidates 
        WHERE resume_data->'skills' @> '["Python", "JavaScript"]'
        """,
        "api_endpoint": "/search/candidates",
        "payload": {
            "filters": {
                "skills": ["Python", "JavaScript"],
                "skills_match_type": "all"  # all skills must be present
            }
        }
    },
    {
        "name": "Experience Path Query",
        "description": "Complex nested JSONB query for career progression",
        "query_type": "experience_path",
        "example_sql": """
        SELECT * FROM candidates 
        WHERE resume_data->'experience' @> '[{"company": "TechCorp", "department": "Engineering"}]'
        """,
        "api_endpoint": "/search/candidates",
        "payload": {
            "filters": {
                "experience": {
                    "companies": ["TechCorp"],
                    "departments": ["Engineering"]
                }
            }
        }
    },
    {
        "name": "Skills Proficiency Search",
        "description": "Search with JSONB nested object filtering",
        "query_type": "skills_proficiency",
        "example_sql": """
        SELECT * FROM candidates 
        WHERE resume_data->'skills_detail'->'Python'->>'level' = 'Expert'
        """,
        "api_endpoint": "/search/advanced",
        "payload": {
            "filters": {
                "skills_detail": {
                    "Python": {"level": "Expert"},
                    "min_proficiency": "Advanced"
                }
            }
        }
    },
    {
        "name": "Location and Experience Combination",
        "description": "Multi-field JSONB search with geographical and experience filters",
        "query_type": "geo_experience",
        "example_sql": """
        SELECT * FROM candidates 
        WHERE resume_data->>'location' ILIKE '%San Francisco%'
        AND (resume_data->'metadata'->>'years_experience')::int >= 5
        """,
        "api_endpoint": "/search/candidates", 
        "payload": {
            "filters": {
                "location": "San Francisco",
                "experience_years": {"min": 5}
            }
        }
    },
    {
        "name": "Company Network Analysis",
        "description": "Find candidates who worked at multiple specific companies",
        "query_type": "company_network",
        "example_sql": """
        SELECT * FROM candidates 
        WHERE EXISTS (
            SELECT 1 FROM jsonb_array_elements(resume_data->'experience') AS exp
            WHERE exp->>'company' IN ('TechCorp', 'DataDyne Solutions')
        )
        """,
        "api_endpoint": "/search/network",
        "payload": {
            "companies": ["TechCorp", "DataDyne Solutions"],
            "network_type": "company_overlap"
        }
    },
    {
        "name": "Skills Intersection Query",
        "description": "Advanced JSONB array intersection for skill matching",
        "query_type": "skills_intersection", 
        "example_sql": """
        SELECT *, 
               jsonb_array_length(
                   jsonb_array_intersect(
                       resume_data->'skills', 
                       '["Python", "AWS", "Docker"]'::jsonb
                   )
               ) as skill_matches
        FROM candidates
        WHERE jsonb_array_length(
            jsonb_array_intersect(
                resume_data->'skills', 
                '["Python", "AWS", "Docker"]'::jsonb
            )
        ) >= 2
        """,
        "api_endpoint": "/search/candidates",
        "payload": {
            "filters": {
                "skills": ["Python", "AWS", "Docker"],
                "skills_match_type": "intersection",
                "min_skill_matches": 2
            },
            "include_score_breakdown": True
        }
    }
]

print(f"\n🔍 Executing {len(jsonb_query_examples)} JSONB query demonstrations:")

jsonb_results = []
query_performance_metrics = []

for i, query_example in enumerate(jsonb_query_examples, 1):
    print(f"\n{i}. {query_example['name']}")
    print(f"   Description: {query_example['description']}")
    print(f"   Query Type: {query_example['query_type']}")
    
    # Display the SQL example
    sql_display = f"""
    <div style="background-color: #f8f9fa; padding: 12px; border-radius: 8px; border-left: 4px solid #0d6efd; margin: 10px 0;">
        <h5 style="margin-top: 0; color: #0d6efd;">📝 Example SQL Query</h5>
        <pre style="background-color: #e9ecef; padding: 10px; border-radius: 5px; overflow-x: auto;"><code>{query_example['example_sql'].strip()}</code></pre>
    </div>
    """
    display(HTML(sql_display))
    
    try:
        # Execute the API call
        endpoint = query_example['api_endpoint']
        payload = query_example['payload']
        
        start_time = time.time()
        
        # Use POST for advanced queries, GET for simple ones
        if 'advanced' in endpoint or len(str(payload)) > 200:
            result = api_client.post(endpoint, json=payload)
        else:
            result = api_client.get(endpoint, params=payload)
        
        response_time = time.time() - start_time
        
        # Process results
        jsonb_result = {
            "query_name": query_example['name'],
            "query_type": query_example['query_type'],
            "endpoint": endpoint,
            "success": result["success"],
            "status_code": result["status_code"],
            "response_time": response_time,
            "timestamp": datetime.now()
        }
        
        if result["success"] and result["data"]:
            jsonb_result["data"] = result["data"]
            results_count = len(result["data"].get("results", []))
            total_count = result["data"].get("total", results_count)
            
            print(f"   ✅ Success: {results_count} results returned (total: {total_count})")
            print(f"   ⏱️ Response time: {response_time:.3f}s")
            
            # Performance metrics for this query type
            query_performance_metrics.append({
                "query_type": query_example['query_type'],
                "response_time": response_time,
                "results_count": results_count,
                "total_count": total_count,
                "complexity": "high" if "advanced" in endpoint else "medium"
            })
            
            # Show sample results with JSONB-specific details
            if result["data"].get("results") and len(result["data"]["results"]) > 0:
                sample_result = result["data"]["results"][0]
                
                # Extract JSONB-relevant fields
                jsonb_details = {}
                if isinstance(sample_result, dict):
                    jsonb_details["candidate_id"] = sample_result.get("id", "N/A")
                    jsonb_details["name"] = sample_result.get("name", "N/A")
                    
                    # Skills handling
                    skills = sample_result.get("skills", [])
                    if isinstance(skills, list) and len(skills) > 0:
                        jsonb_details["skills_count"] = len(skills)
                        jsonb_details["sample_skills"] = skills[:3]
                    
                    # Experience handling
                    experience = sample_result.get("experience", [])
                    if isinstance(experience, list) and len(experience) > 0:
                        current_role = experience[0]
                        jsonb_details["current_company"] = current_role.get("company", "N/A")
                        jsonb_details["current_department"] = current_role.get("department", "N/A")
                    
                    # Score handling
                    jsonb_details["relevance_score"] = sample_result.get("score", sample_result.get("relevance_score", "N/A"))
                
                print(f"   📋 Sample result: {jsonb_details.get('name', 'N/A')} (Score: {jsonb_details.get('relevance_score', 'N/A')})")
                print(f"      Company: {jsonb_details.get('current_company', 'N/A')}, Dept: {jsonb_details.get('current_department', 'N/A')}")
                print(f"      Skills: {jsonb_details.get('skills_count', 0)} total, sample: {jsonb_details.get('sample_skills', [])}")
        
        else:
            print(f"   ❌ Failed: {result.get('status_code', 'Unknown')} - {result.get('error', 'No error details')}")
            query_performance_metrics.append({
                "query_type": query_example['query_type'],
                "response_time": response_time,
                "results_count": 0,
                "total_count": 0,
                "complexity": "failed"
            })
        
        jsonb_results.append(jsonb_result)
        
        # Small delay between queries
        time.sleep(0.1)
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        jsonb_results.append({
            "query_name": query_example['name'],
            "query_type": query_example['query_type'],
            "endpoint": query_example.get('api_endpoint', 'unknown'),
            "success": False,
            "status_code": None,
            "response_time": 0,
            "timestamp": datetime.now(),
            "error": str(e)
        })

# Analyze JSONB query performance
print(f"\n📊 JSONB Query Performance Analysis:")
successful_queries = sum(1 for r in jsonb_results if r['success'])
total_queries = len(jsonb_results)
success_rate = (successful_queries / total_queries * 100) if total_queries > 0 else 0

print(f"   Total JSONB queries tested: {total_queries}")
print(f"   Successful queries: {successful_queries}")
print(f"   Success rate: {success_rate:.1f}%")

if query_performance_metrics:
    avg_response_time = sum(q['response_time'] for q in query_performance_metrics) / len(query_performance_metrics)
    complex_queries = [q for q in query_performance_metrics if q['complexity'] == 'high']
    simple_queries = [q for q in query_performance_metrics if q['complexity'] == 'medium']
    
    print(f"   Average response time: {avg_response_time:.3f}s")
    
    if complex_queries:
        complex_avg = sum(q['response_time'] for q in complex_queries) / len(complex_queries)
        print(f"   Complex queries avg: {complex_avg:.3f}s")
    
    if simple_queries:
        simple_avg = sum(q['response_time'] for q in simple_queries) / len(simple_queries)
        print(f"   Simple queries avg: {simple_avg:.3f}s")

print("\n✅ JSONB query demonstrations completed!")

### 🔬 JSONB Query Demonstrations

Showcasing PostgreSQL JSONB queries and their performance in the search system.

In [ ]:
print("📊 Advanced Search Results Visualization...")

# Create comprehensive visualization of sophisticated search results
if sophisticated_search_results and search_performance_data:
    
    # Create search results DataFrame for visualization
    search_viz_df = create_search_visualization_data(sophisticated_search_results)
    
    # Create advanced search analytics dashboard
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Search Response Times by Scenario',
            'Search Success Rate by Type', 
            'Results Count Distribution',
            'Search Performance vs Results',
            'Search Scoring Analysis',
            'Search Type Comparison'
        ),
        specs=[[{"secondary_y": False}, {"type": "pie"}],
               [{"type": "histogram"}, {"type": "scatter"}],
               [{"type": "bar"}, {"type": "box"}]]
    )
    
    # 1. Response times by scenario
    scenario_names = [r['scenario'] for r in sophisticated_search_results if 'response_time' in r]
    response_times = [r['response_time'] for r in sophisticated_search_results if 'response_time' in r]
    
    fig.add_trace(
        go.Bar(
            x=list(range(len(scenario_names))),
            y=response_times,
            text=scenario_names,
            textangle=45,
            name="Response Time",
            marker_color='lightblue'
        ),
        row=1, col=1
    )
    
    # 2. Success rate pie chart
    successful_searches = sum(1 for r in sophisticated_search_results if r.get('success', False))
    failed_searches = len(sophisticated_search_results) - successful_searches
    
    fig.add_trace(
        go.Pie(
            labels=['Successful', 'Failed'],
            values=[successful_searches, failed_searches],
            marker_colors=['lightgreen', 'lightcoral']
        ),
        row=1, col=2
    )
    
    # 3. Results count distribution
    if search_performance_data:
        results_counts = [d['results_count'] for d in search_performance_data]
        fig.add_trace(
            go.Histogram(x=results_counts, nbinsx=10, name="Results Count"),
            row=2, col=1
        )
    
    # 4. Performance vs results scatter
    if search_performance_data:
        perf_response_times = [d['response_time'] for d in search_performance_data]
        perf_results_counts = [d['results_count'] for d in search_performance_data]
        
        fig.add_trace(
            go.Scatter(
                x=perf_results_counts,
                y=perf_response_times,
                mode='markers',
                marker=dict(size=10, opacity=0.7),
                name="Performance vs Results"
            ),
            row=2, col=2
        )
    
    # 5. Search scoring analysis (if score data available)
    if search_viz_df is not None and not search_viz_df.empty and 'score' in search_viz_df.columns:
        search_scores = search_viz_df['score'].dropna()
        if len(search_scores) > 0:
            fig.add_trace(
                go.Bar(
                    x=search_viz_df['search_type'].unique(),
                    y=search_viz_df.groupby('search_type')['score'].mean(),
                    name="Average Score by Type"
                ),
                row=3, col=1
            )
    
    # 6. Search type comparison (box plot of response times)
    if len(sophisticated_search_results) > 0:
        search_types = [r['scenario'] for r in sophisticated_search_results]
        search_response_times = [r['response_time'] for r in sophisticated_search_results]
        
        fig.add_trace(
            go.Box(
                y=search_response_times,
                name="Response Time Distribution"
            ),
            row=3, col=2
        )
    
    fig.update_layout(
        height=1000,
        title_text="Sophisticated Search Functionality - Comprehensive Analytics",
        showlegend=False
    )
    
    fig.show()
    
    # Create detailed search feature analysis
    print(f"\n🎯 Search Feature Analysis:")
    
    # Analyze different search types
    search_type_analysis = {}
    for result in sophisticated_search_results:
        scenario = result['scenario']
        if scenario not in search_type_analysis:
            search_type_analysis[scenario] = {
                'attempts': 0,
                'successes': 0,
                'total_response_time': 0,
                'results_returned': 0
            }
        
        search_type_analysis[scenario]['attempts'] += 1
        if result.get('success', False):
            search_type_analysis[scenario]['successes'] += 1
            search_type_analysis[scenario]['total_response_time'] += result.get('response_time', 0)
            
            # Count results if available
            if 'data' in result and result['data']:
                results_count = len(result['data'].get('results', []))
                search_type_analysis[scenario]['results_returned'] += results_count
    
    # Display search type analysis
    feature_analysis_html = """
    <div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; margin: 15px 0;">
        <h4>🔍 Search Feature Performance Analysis</h4>
        <table style="width: 100%; border-collapse: collapse;">
            <thead>
                <tr style="background-color: #e9ecef;">
                    <th style="padding: 10px; text-align: left; border: 1px solid #ddd;">Search Type</th>
                    <th style="padding: 10px; text-align: center; border: 1px solid #ddd;">Success Rate</th>
                    <th style="padding: 10px; text-align: center; border: 1px solid #ddd;">Avg Response Time</th>
                    <th style="padding: 10px; text-align: center; border: 1px solid #ddd;">Results Returned</th>
                    <th style="padding: 10px; text-align: center; border: 1px solid #ddd;">Status</th>
                </tr>
            </thead>
            <tbody>
    """
    
    for search_type, analysis in search_type_analysis.items():
        success_rate = (analysis['successes'] / analysis['attempts'] * 100) if analysis['attempts'] > 0 else 0
        avg_response_time = (analysis['total_response_time'] / analysis['successes']) if analysis['successes'] > 0 else 0
        
        status_icon = "✅" if success_rate >= 80 else "⚠️" if success_rate >= 50 else "❌"
        
        feature_analysis_html += f"""
                <tr>
                    <td style="padding: 8px; border: 1px solid #ddd;">{search_type}</td>
                    <td style="padding: 8px; border: 1px solid #ddd; text-align: center;">{success_rate:.1f}%</td>
                    <td style="padding: 8px; border: 1px solid #ddd; text-align: center;">{avg_response_time:.3f}s</td>
                    <td style="padding: 8px; border: 1px solid #ddd; text-align: center;">{analysis['results_returned']}</td>
                    <td style="padding: 8px; border: 1px solid #ddd; text-align: center;">{status_icon}</td>
                </tr>
        """
    
    feature_analysis_html += """
            </tbody>
        </table>
    </div>
    """
    
    display(HTML(feature_analysis_html))
    
    # Search capabilities summary
    capabilities_summary = f"""
    <div style="background: linear-gradient(135deg, #4CAF50 0%, #45a049 100%); color: white; padding: 20px; border-radius: 15px; margin: 15px 0;">
        <h3 style="margin-top: 0; text-align: center;">🚀 Advanced Search Capabilities Demonstrated</h3>
        
        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 15px; margin: 15px 0;">
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h4>📊 Multi-Criteria Search</h4>
                <p>✅ Skills, experience, location filtering<br>
                ✅ Weighted scoring algorithms<br>
                ✅ Dynamic result ranking</p>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h4>🔍 Similarity Matching</h4>
                <p>✅ Profile similarity analysis<br>
                ✅ Career path comparison<br>
                ✅ Skills overlap detection</p>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h4>🤝 Network Discovery</h4>
                <p>✅ Colleague identification<br>
                ✅ Company overlap analysis<br>
                ✅ Professional connections</p>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h4>🧠 AI-Powered Search</h4>
                <p>✅ Natural language queries<br>
                ✅ Intent understanding<br>
                ✅ Smart result enhancement</p>
            </div>
        </div>
        
        <div style="text-align: center; margin-top: 15px; padding: 15px; background: rgba(255,255,255,0.1); border-radius: 10px;">
            <h4>📈 Performance Metrics</h4>
            <p>Search scenarios tested: {len(sophisticated_search_results)} | Success rate: {successful_searches/len(sophisticated_search_results)*100:.1f}% | Avg response: {sum(r.get('response_time', 0) for r in sophisticated_search_results)/len(sophisticated_search_results):.3f}s</p>
        </div>
    </div>
    """
    
    display(HTML(capabilities_summary))
    
else:
    print("⚠️ No sophisticated search results available for visualization")

print("\n📊 Advanced search visualization completed!")

In [ ]:
print("🎯 Testing Sophisticated Search Scenarios...")

# Get comprehensive search scenarios
search_scenarios = create_test_search_scenarios()
sophisticated_search_results = []
search_performance_data = []

print(f"\n🔍 Executing {len(search_scenarios)} sophisticated search scenarios:")

for i, scenario in enumerate(search_scenarios, 1):
    print(f"\n{i}. {scenario['name']}")
    print(f"   Description: {scenario['description']}")
    
    try:
        # Prepare the request
        endpoint = scenario['endpoint']
        payload = scenario['payload'].copy()
        
        # Fill in candidate IDs for similar/colleague searches
        if 'candidate_id' in payload and payload['candidate_id'] == 'will_be_filled':
            if uploaded_candidate_ids:
                payload['candidate_id'] = uploaded_candidate_ids[0]  # Use first uploaded candidate
            else:
                payload['candidate_id'] = 'demo_candidate_001'  # Fallback
        
        # Make the API request
        start_time = time.time()
        
        if scenario.get('method') == 'POST':
            result = api_client.post(endpoint, json=payload)
        else:
            result = api_client.get(endpoint, params=payload)
        
        response_time = time.time() - start_time
        
        # Process results
        search_result = {
            "scenario": scenario['name'],
            "endpoint": endpoint,
            "success": result["success"],
            "status_code": result["status_code"],
            "response_time": response_time,
            "timestamp": datetime.now(),
            "search_type": scenario['name']
        }
        
        if result["success"] and result["data"]:
            search_result["data"] = result["data"]
            results_count = len(result["data"].get("results", []))
            total_count = result["data"].get("total", results_count)
            
            print(f"   ✅ Success: {results_count} results returned (total: {total_count})")
            print(f"   ⏱️ Response time: {response_time:.3f}s")
            
            # Extract performance metrics
            search_performance_data.append({
                "scenario": scenario['name'],
                "response_time": response_time,
                "results_count": results_count,
                "total_count": total_count,
                "success": True
            })
            
            # Display sample results if available
            if result["data"].get("results"):
                sample_result = result["data"]["results"][0]
                if isinstance(sample_result, dict):
                    sample_info = {
                        "name": sample_result.get("name", "N/A"),
                        "score": sample_result.get("score", sample_result.get("relevance_score", "N/A")),
                        "department": "N/A",
                        "company": "N/A"
                    }
                    
                    # Try to extract department and company from experience
                    experience = sample_result.get("experience", [])
                    if experience and len(experience) > 0:
                        sample_info["department"] = experience[0].get("department", "N/A")
                        sample_info["company"] = experience[0].get("company", "N/A")
                    
                    print(f"   📋 Sample result: {sample_info['name']} (Score: {sample_info['score']}) - {sample_info['department']} at {sample_info['company']}")
            
            # Check for special features
            expected_features = scenario.get('expected_features', [])
            for feature in expected_features:
                if feature in str(result["data"]).lower():
                    print(f"   🎯 Feature confirmed: {feature}")
        
        else:
            print(f"   ❌ Failed: {result.get('status_code', 'Unknown')} - {result.get('error', 'No error details')}")
            search_performance_data.append({
                "scenario": scenario['name'],
                "response_time": response_time,
                "results_count": 0,
                "total_count": 0,
                "success": False
            })
        
        sophisticated_search_results.append(search_result)
        
        # Small delay between requests to avoid overwhelming the API
        time.sleep(0.1)
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        sophisticated_search_results.append({
            "scenario": scenario['name'],
            "endpoint": scenario.get('endpoint', 'unknown'),
            "success": False,
            "status_code": None,
            "response_time": 0,
            "timestamp": datetime.now(),
            "error": str(e)
        })

print(f"\n📊 Sophisticated Search Testing Summary:")
successful_scenarios = sum(1 for r in sophisticated_search_results if r['success'])
total_scenarios = len(sophisticated_search_results)
success_rate = (successful_scenarios / total_scenarios * 100) if total_scenarios > 0 else 0

print(f"   Total scenarios tested: {total_scenarios}")
print(f"   Successful scenarios: {successful_scenarios}")
print(f"   Success rate: {success_rate:.1f}%")

# Analyze search performance
if search_performance_data:
    performance_analysis = analyze_search_performance(sophisticated_search_results)
    
    print(f"\n⚡ Search Performance Analysis:")
    print(f"   Average response time: {performance_analysis.get('avg_response_time', 0):.3f}s")
    print(f"   Fastest search: {performance_analysis.get('min_response_time', 0):.3f}s")
    print(f"   Slowest search: {performance_analysis.get('max_response_time', 0):.3f}s")
    print(f"   Searches under 1s: {performance_analysis.get('searches_under_1s', 0)}/{performance_analysis.get('total_searches', 0)}")
    print(f"   Searches under 2s: {performance_analysis.get('searches_under_2s', 0)}/{performance_analysis.get('total_searches', 0)}")

print("\n✅ Sophisticated search scenario testing completed!")

In [ ]:
# Import enhanced search testing module
from enhanced_search_testing import (
    generate_realistic_candidate,
    create_test_search_scenarios,
    create_search_visualization_data,
    analyze_search_performance,
    COMPANIES,
    DEPARTMENTS,
    SKILLS_BY_CATEGORY
)

print("🔍 Enhanced Search Testing - Creating Realistic Test Dataset...")

# Generate realistic candidate dataset
print("\n1. Generating Realistic Candidate Profiles")
test_candidates = []
num_candidates = 50  # Generate 50 realistic candidates

for i in range(1, num_candidates + 1):
    candidate = generate_realistic_candidate(i)
    test_candidates.append(candidate)

print(f"✅ Generated {len(test_candidates)} realistic candidate profiles")

# Display sample candidate
sample_candidate = test_candidates[0]
sample_display = f"""
<div style="background-color: #f8f9fa; padding: 15px; border-radius: 10px; border-left: 4px solid #007acc;">
    <h4>📋 Sample Generated Candidate Profile</h4>
    <p><strong>Name:</strong> {sample_candidate['name']}</p>
    <p><strong>Location:</strong> {sample_candidate['location']}</p>
    <p><strong>Experience:</strong> {sample_candidate['metadata']['years_experience']} years</p>
    <p><strong>Current Role:</strong> {sample_candidate['experience'][0]['position']} at {sample_candidate['experience'][0]['company']}</p>
    <p><strong>Department:</strong> {sample_candidate['experience'][0]['department']} - {sample_candidate['experience'][0]['desk']}</p>
    <p><strong>Skills:</strong> {', '.join(sample_candidate['skills'][:5])}{'...' if len(sample_candidate['skills']) > 5 else ''}</p>
</div>
"""
display(HTML(sample_display))

# Upload test candidates to API (simulate bulk upload)
print("\n2. Uploading Test Candidates to API")
uploaded_candidate_ids = []

# For demo purposes, we'll upload first 10 candidates
for i, candidate in enumerate(test_candidates[:10]):
    try:
        # Convert candidate to resume format for upload
        resume_data = {
            "candidate_data": candidate,
            "source": "test_generation",
            "parse_status": "completed"
        }
        
        result = api_client.post("/resumes/bulk-upload", json=resume_data)
        
        if result["success"] and result["data"]:
            candidate_id = result["data"].get("id")
            if candidate_id:
                uploaded_candidate_ids.append(candidate_id)
                print(f"✅ Uploaded candidate {i+1}: ID {candidate_id}")
        else:
            # If bulk upload doesn't exist, use individual upload simulation
            print(f"📝 Simulated upload for candidate {i+1}: {candidate['name']}")
            uploaded_candidate_ids.append(f"sim_{i+1}")
            
    except Exception as e:
        print(f"⚠️ Upload simulation for candidate {i+1}: {str(e)}")
        uploaded_candidate_ids.append(f"sim_{i+1}")

print(f"\n📊 Dataset Summary:")
print(f"   Total candidates generated: {len(test_candidates)}")
print(f"   Candidates uploaded/simulated: {len(uploaded_candidate_ids)}")

# Analyze the generated dataset
companies_count = {}
departments_count = {}
skills_count = {}

for candidate in test_candidates:
    # Count companies
    current_company = candidate['experience'][0]['company']
    companies_count[current_company] = companies_count.get(current_company, 0) + 1
    
    # Count departments
    current_dept = candidate['experience'][0]['department']
    departments_count[current_dept] = departments_count.get(current_dept, 0) + 1
    
    # Count skills
    for skill in candidate['skills']:
        skills_count[skill] = skills_count.get(skill, 0) + 1

# Display dataset analytics
print(f"\n📈 Dataset Analytics:")
print(f"   Companies represented: {len(companies_count)}")
print(f"   Departments represented: {len(departments_count)}")
print(f"   Unique skills: {len(skills_count)}")
print(f"   Top companies: {sorted(companies_count.items(), key=lambda x: x[1], reverse=True)[:3]}")
print(f"   Top departments: {sorted(departments_count.items(), key=lambda x: x[1], reverse=True)[:3]}")
print(f"   Top skills: {sorted(skills_count.items(), key=lambda x: x[1], reverse=True)[:5]}")

print("\n✅ Realistic test dataset created successfully!")

---

## 🔍 Enhanced Search Functionality Testing

Testing sophisticated search capabilities including multi-criteria search, similar profile matching, colleague discovery, and AI-powered search with comprehensive test datasets.

---

## 🎯 Summary and Next Steps

This comprehensive testing notebook has covered:

### ✅ Completed Tests
1. **API Health Checks** - Verified basic connectivity and endpoint availability
2. **Authentication Flow** - Tested login, token management, and protected endpoints
3. **Resume Upload** - Tested file upload with various formats and Claude AI parsing
4. **Basic Search Functionality** - Initial search testing with different query types
5. **🆕 Enhanced Search Capabilities** - Sophisticated search scenarios including:
   - **Multi-Criteria Candidate Search** with weighted scoring algorithms
   - **Similar Profile Matching** using AI-powered similarity analysis
   - **Colleague Discovery** with company/department overlap detection
   - **Smart Natural Language Search** with query enhancement
   - **Advanced Multi-Factor Search** with boosting and score breakdown
   - **JSONB Query Demonstrations** showcasing PostgreSQL advanced queries
6. **Performance Testing** - Load testing with concurrent requests and stress testing
7. **Data Visualization** - Rich charts and analytics for all test results

### 🚀 Advanced Search Features Demonstrated
- **🎯 Multi-Criteria Search**: Skills, experience, location filtering with weighted scoring
- **🤖 AI-Powered Similarity**: Profile matching using career path and skills analysis
- **🤝 Professional Networks**: Colleague discovery with temporal overlap analysis
- **🧠 Natural Language**: Smart search with query understanding and enhancement
- **🔍 JSONB Queries**: Advanced PostgreSQL JSONB operations for complex filtering
- **📊 Scoring Algorithms**: Transparent relevance scoring with breakdown explanations
- **⚡ Performance**: Sub-2-second response times for complex searches

### 📊 Key Features
- **Interactive Testing** - Real-time API testing with immediate feedback
- **Performance Metrics** - Detailed response time and success rate analysis
- **Visual Analytics** - Comprehensive charts and dashboards including search-specific visualizations
- **Automated Reporting** - Generated markdown reports for documentation
- **Error Handling** - Robust error detection and reporting
- **🆕 Search Analytics** - Advanced search performance analysis and scoring visualization
- **🆕 JSONB Demonstrations** - Real-world PostgreSQL JSONB query examples

### 🚀 Usage Instructions
1. Ensure API server is running at `http://localhost:8000`
2. Ensure PostgreSQL database is configured and accessible
3. Run notebook cells sequentially
4. Monitor test results and visualizations
5. Review generated test report
6. Use insights for API improvement

### 🔧 Customization
- Modify `TEST_TIMEOUT` for different response time requirements
- Adjust `CONCURRENT_REQUESTS` for different load testing scenarios
- Update test data in helper functions for specific testing needs
- Extend visualization functions for custom charts
- **🆕 Customize Search Scenarios**: Modify `enhanced_search_testing.py` for domain-specific search patterns
- **🆕 JSONB Query Expansion**: Add more complex PostgreSQL JSONB query examples

### 📈 Next Steps
- Integrate with CI/CD pipeline for automated testing
- Add more sophisticated test scenarios
- Implement test data factories for larger datasets
- Create alerts based on performance thresholds
- **🆕 Search Enhancement**: Implement A/B testing for search algorithms
- **🆕 AI Integration**: Expand natural language search capabilities
- **🆕 Performance Optimization**: Implement search result caching strategies

### 🔬 Technical Highlights
- **Database Technology**: PostgreSQL with advanced JSONB operations
- **Search Architecture**: Multi-layered search with scoring algorithms
- **AI Integration**: Claude API for resume parsing and query enhancement
- **Performance**: Concurrent request handling with sub-2s response targets
- **Scalability**: Designed for enterprise-level candidate databases

---

**Happy Testing! 🧪✨**

*Enhanced with sophisticated search capabilities and comprehensive PostgreSQL JSONB demonstrations*

In [ ]:
print("⚡ Testing API Performance...")

async def make_concurrent_requests(endpoint: str, num_requests: int = 10, params: dict = None):
    """Make concurrent requests to test load handling"""
    results = []
    
    async def make_request(request_id: int):
        start_time = time.time()
        try:
            result = await api_client.async_get(endpoint, params=params or {})
            return {
                "request_id": request_id,
                "success": result["success"],
                "status_code": result["status_code"],
                "response_time": result["response_time"],
                "timestamp": datetime.now(),
                "endpoint": endpoint
            }
        except Exception as e:
            return {
                "request_id": request_id,
                "success": False,
                "status_code": None,
                "response_time": time.time() - start_time,
                "timestamp": datetime.now(),
                "endpoint": endpoint,
                "error": str(e)
            }
    
    # Execute concurrent requests
    tasks = [make_request(i) for i in range(num_requests)]
    results = await asyncio.gather(*tasks)
    
    return results

# Performance test endpoints
perf_endpoints = [
    {
        "endpoint": "/health",
        "name": "Health check",
        "params": None,
        "concurrent_requests": 20
    },
    {
        "endpoint": "/search",
        "name": "Basic search",
        "params": {"q": "engineer"},
        "concurrent_requests": 15
    },
    {
        "endpoint": "/resumes",
        "name": "Resume list",
        "params": {"limit": 10},
        "concurrent_requests": 10
    }
]

performance_results = []

print("\n1. Testing Concurrent Request Handling")
for test_config in perf_endpoints:
    print(f"\nTesting {test_config['name']} with {test_config['concurrent_requests']} concurrent requests...")
    
    # Adjust endpoint for base URL if needed
    endpoint = test_config['endpoint']
    if endpoint in ["/health", "/readiness"]:
        endpoint = f"{API_BASE_URL}{endpoint}"
    
    try:
        start_time = time.time()
        results = await make_concurrent_requests(
            endpoint,
            test_config['concurrent_requests'],
            test_config['params']
        )
        total_time = time.time() - start_time
        
        # Analyze results
        successful_requests = sum(1 for r in results if r['success'])
        failed_requests = len(results) - successful_requests
        avg_response_time = sum(r['response_time'] for r in results) / len(results)
        max_response_time = max(r['response_time'] for r in results)
        min_response_time = min(r['response_time'] for r in results)
        
        # Store performance data
        perf_data = {
            "test": f"Concurrent {test_config['name']}",
            "endpoint": test_config['endpoint'],
            "concurrent_requests": test_config['concurrent_requests'],
            "successful_requests": successful_requests,
            "failed_requests": failed_requests,
            "success_rate": (successful_requests / len(results)) * 100,
            "total_time": total_time,
            "avg_response_time": avg_response_time,
            "max_response_time": max_response_time,
            "min_response_time": min_response_time,
            "requests_per_second": len(results) / total_time,
            "timestamp": datetime.now()
        }
        
        performance_results.append(perf_data)
        
        # Add individual results
        for result in results:
            result['test_name'] = test_config['name']
        
        print(f"✅ {test_config['name']}:")
        print(f"   Success rate: {perf_data['success_rate']:.1f}%")
        print(f"   Avg response time: {avg_response_time:.3f}s")
        print(f"   Requests/second: {perf_data['requests_per_second']:.1f}")
        print(f"   Total time: {total_time:.3f}s")
        
    except Exception as e:
        print(f"❌ {test_config['name']}: Error - {str(e)}")
        performance_results.append({
            "test": f"Concurrent {test_config['name']}",
            "endpoint": test_config['endpoint'],
            "error": str(e),
            "timestamp": datetime.now()
        })

# Stress testing with increasing load
print("\n2. Stress Testing with Increasing Load")
stress_loads = [5, 10, 20, 30, 50]
stress_results = []

for load in stress_loads:
    print(f"Testing with {load} concurrent requests...")
    
    try:
        start_time = time.time()
        results = await make_concurrent_requests(f"{API_BASE_URL}/health", load)
        total_time = time.time() - start_time
        
        successful = sum(1 for r in results if r['success'])
        avg_time = sum(r['response_time'] for r in results) / len(results)
        
        stress_data = {
            "load": load,
            "success_rate": (successful / len(results)) * 100,
            "avg_response_time": avg_time,
            "requests_per_second": len(results) / total_time,
            "total_time": total_time
        }
        
        stress_results.append(stress_data)
        
        print(f"   Success: {stress_data['success_rate']:.1f}%, "
              f"Avg time: {avg_time:.3f}s, "
              f"RPS: {stress_data['requests_per_second']:.1f}")
        
    except Exception as e:
        print(f"❌ Load {load}: Error - {str(e)}")

# Visualize performance results
print("\n📊 Performance Test Visualization")

if performance_results and stress_results:
    # Create performance dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Response Time by Endpoint',
            'Success Rate vs Load',
            'Requests per Second',
            'Load Testing Results'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Response time by endpoint
    endpoints = [p['endpoint'] for p in performance_results if 'avg_response_time' in p]
    response_times = [p['avg_response_time'] for p in performance_results if 'avg_response_time' in p]
    
    fig.add_trace(
        go.Bar(x=endpoints, y=response_times, name="Response Time"),
        row=1, col=1
    )
    
    # Success rate vs load (stress test)
    loads = [s['load'] for s in stress_results]
    success_rates = [s['success_rate'] for s in stress_results]
    
    fig.add_trace(
        go.Scatter(x=loads, y=success_rates, mode='lines+markers', name="Success Rate"),
        row=1, col=2
    )
    
    # Requests per second
    rps_values = [p['requests_per_second'] for p in performance_results if 'requests_per_second' in p]
    
    fig.add_trace(
        go.Bar(x=endpoints, y=rps_values, name="RPS"),
        row=2, col=1
    )
    
    # Load testing timeline
    avg_times = [s['avg_response_time'] for s in stress_results]
    
    fig.add_trace(
        go.Scatter(x=loads, y=avg_times, mode='lines+markers', name="Avg Response Time"),
        row=2, col=2
    )
    
    fig.update_layout(height=800, title_text="API Performance Test Results", showlegend=False)
    fig.show()
    
    # Performance summary
    if performance_results:
        best_perf = min(performance_results, key=lambda x: x.get('avg_response_time', float('inf')))
        worst_perf = max(performance_results, key=lambda x: x.get('avg_response_time', 0))
        
        perf_summary = f"""
        <div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; margin: 10px 0;">
            <h4>⚡ Performance Summary</h4>
            <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px;">
                <div>
                    <h5 style="color: #28a745;">🏆 Best Performance</h5>
                    <p><strong>Endpoint:</strong> {best_perf.get('endpoint', 'N/A')}</p>
                    <p><strong>Avg Response:</strong> {best_perf.get('avg_response_time', 0):.3f}s</p>
                    <p><strong>Success Rate:</strong> {best_perf.get('success_rate', 0):.1f}%</p>
                </div>
                <div>
                    <h5 style="color: #dc3545;">🐌 Slowest Performance</h5>
                    <p><strong>Endpoint:</strong> {worst_perf.get('endpoint', 'N/A')}</p>
                    <p><strong>Avg Response:</strong> {worst_perf.get('avg_response_time', 0):.3f}s</p>
                    <p><strong>Success Rate:</strong> {worst_perf.get('success_rate', 0):.1f}%</p>
                </div>
            </div>
        </div>
        """
        display(HTML(perf_summary))

print(f"\n⚡ Performance testing completed - {len(performance_results)} endpoint tests, {len(stress_results)} stress tests")

---

## 📊 Data Visualization and Analytics

Comprehensive visualization of test results and API analytics.

In [ ]:
print("📊 Creating Comprehensive Data Visualizations...")

# Compile all test results
all_test_results = {
    "Health Checks": health_results,
    "Authentication": auth_results,
    "Resume Upload": upload_results,
    "Search Functionality": search_results,
    "Performance Tests": performance_results
}

# Create comprehensive analytics dashboard
def create_analytics_dashboard():
    """Create a comprehensive analytics dashboard"""
    
    # Combine all results
    combined_results = []
    for category, results in all_test_results.items():
        for result in results:
            result_copy = result.copy()
            result_copy['category'] = category
            combined_results.append(result_copy)
    
    if not combined_results:
        print("⚠️ No test results available for visualization")
        return
    
    df = pd.DataFrame(combined_results)
    
    # Create comprehensive dashboard
    fig = make_subplots(
        rows=3, cols=3,
        subplot_titles=(
            'Test Results by Category',
            'Success Rate by Category',
            'Response Time Distribution',
            'Timeline Analysis',
            'Status Code Distribution',
            'Performance Over Time',
            'Test Volume by Category',
            'Error Analysis',
            'API Health Score'
        ),
        specs=[[{"type": "bar"}, {"type": "pie"}, {"type": "histogram"}],
               [{"type": "scatter"}, {"type": "bar"}, {"type": "scatter"}],
               [{"type": "bar"}, {"type": "bar"}, {"type": "indicator"}]]
    )
    
    # 1. Test results by category
    category_success = df.groupby('category')['success'].agg(['sum', 'count']).reset_index()
    category_success['success_rate'] = (category_success['sum'] / category_success['count']) * 100
    
    fig.add_trace(
        go.Bar(
            x=category_success['category'],
            y=category_success['success_rate'],
            name="Success Rate",
            marker_color='lightblue'
        ),
        row=1, col=1
    )
    
    # 2. Overall success rate pie chart
    total_success = df['success'].sum()
    total_tests = len(df)
    fig.add_trace(
        go.Pie(
            labels=['Success', 'Failed'],
            values=[total_success, total_tests - total_success],
            marker_colors=['lightgreen', 'lightcoral']
        ),
        row=1, col=2
    )
    
    # 3. Response time distribution
    response_times = df['response_time'].dropna()
    if len(response_times) > 0:
        fig.add_trace(
            go.Histogram(x=response_times, nbinsx=20, name="Response Times"),
            row=1, col=3
        )
    
    # 4. Timeline analysis
    if 'timestamp' in df.columns:
        df_sorted = df.sort_values('timestamp')
        fig.add_trace(
            go.Scatter(
                x=df_sorted.index,
                y=df_sorted['response_time'],
                mode='lines+markers',
                name="Response Time Timeline"
            ),
            row=2, col=1
        )
    
    # 5. Status code distribution
    status_codes = df['status_code'].dropna().astype(str)
    if len(status_codes) > 0:
        status_counts = status_codes.value_counts()
        fig.add_trace(
            go.Bar(x=status_counts.index, y=status_counts.values, name="Status Codes"),
            row=2, col=2
        )
    
    # 6. Performance by category
    if 'response_time' in df.columns:
        category_perf = df.groupby('category')['response_time'].mean().reset_index()
        fig.add_trace(
            go.Scatter(
                x=category_perf['category'],
                y=category_perf['response_time'],
                mode='markers',
                marker_size=15,
                name="Avg Response Time"
            ),
            row=2, col=3
        )
    
    # 7. Test volume by category
    category_counts = df['category'].value_counts()
    fig.add_trace(
        go.Bar(x=category_counts.index, y=category_counts.values, name="Test Count"),
        row=3, col=1
    )
    
    # 8. Error analysis
    error_df = df[df['success'] == False]
    if len(error_df) > 0:
        error_categories = error_df['category'].value_counts()
        fig.add_trace(
            go.Bar(x=error_categories.index, y=error_categories.values, 
                   name="Errors", marker_color='red'),
            row=3, col=2
        )
    
    # 9. API Health Score
    health_score = (total_success / total_tests) * 100 if total_tests > 0 else 0
    avg_response_time = df['response_time'].mean() if 'response_time' in df.columns else 0
    
    # Calculate composite health score
    response_score = max(0, 100 - (avg_response_time * 50))  # Penalty for slow responses
    composite_score = (health_score * 0.7) + (response_score * 0.3)
    
    fig.add_trace(
        go.Indicator(
            mode="gauge+number+delta",
            value=composite_score,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "API Health Score"},
            gauge={
                'axis': {'range': [None, 100]},
                'bar': {'color': "darkblue"},
                'steps': [
                    {'range': [0, 50], 'color': "lightgray"},
                    {'range': [50, 80], 'color': "yellow"},
                    {'range': [80, 100], 'color': "green"}
                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 90
                }
            }
        ),
        row=3, col=3
    )
    
    fig.update_layout(
        height=1200,
        title_text="HR Resume Search MCP API - Comprehensive Test Analytics Dashboard",
        showlegend=False
    )
    
    fig.show()
    
    return df, {
        'total_tests': total_tests,
        'successful_tests': total_success,
        'success_rate': health_score,
        'avg_response_time': avg_response_time,
        'health_score': composite_score
    }

# Create the dashboard
test_df, summary_stats = create_analytics_dashboard()

# Display comprehensive summary
if summary_stats:
    summary_html = f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 15px; margin: 20px 0;">
        <h2 style="margin-top: 0; text-align: center;">🎯 HR Resume Search MCP API Test Summary</h2>
        
        <div style="display: grid; grid-template-columns: repeat(5, 1fr); gap: 20px; margin: 20px 0;">
            <div style="text-align: center; background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 2.5em; font-weight: bold;">{summary_stats['total_tests']}</div>
                <div style="font-size: 0.9em; opacity: 0.9;">Total Tests</div>
            </div>
            <div style="text-align: center; background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 2.5em; font-weight: bold; color: #4CAF50;">{summary_stats['successful_tests']}</div>
                <div style="font-size: 0.9em; opacity: 0.9;">Successful</div>
            </div>
            <div style="text-align: center; background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 2.5em; font-weight: bold; color: #2196F3;">{summary_stats['success_rate']:.1f}%</div>
                <div style="font-size: 0.9em; opacity: 0.9;">Success Rate</div>
            </div>
            <div style="text-align: center; background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 2.5em; font-weight: bold; color: #FF9800;">{summary_stats['avg_response_time']:.3f}s</div>
                <div style="font-size: 0.9em; opacity: 0.9;">Avg Response</div>
            </div>
            <div style="text-align: center; background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 2.5em; font-weight: bold; color: #9C27B0;">{summary_stats['health_score']:.0f}</div>
                <div style="font-size: 0.9em; opacity: 0.9;">Health Score</div>
            </div>
        </div>
        
        <div style="text-align: center; margin-top: 20px; padding: 15px; background: rgba(255,255,255,0.1); border-radius: 10px;">
            <h3 style="margin: 0;">📊 Test Categories Covered</h3>
            <p style="margin: 10px 0;">Health Checks • Authentication • Resume Upload • Search Functionality • Performance Testing</p>
        </div>
    </div>
    """
    
    display(HTML(summary_html))

print(f"\n📊 Analytics dashboard created successfully!")
print(f"📈 Total tests executed: {summary_stats['total_tests'] if summary_stats else 0}")
print(f"✅ Success rate: {summary_stats['success_rate']:.1f}% " if summary_stats else "No summary available")

---

## 📋 Test Report Generation

Generate comprehensive test reports for documentation and sharing.

In [ ]:
print("📋 Generating Comprehensive Test Report...")

def generate_detailed_report():
    """Generate a detailed test report with all metrics"""
    
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Get client metrics
    client_metrics = api_client.get_metrics()
    
    # Calculate detailed statistics
    total_tests = sum(len(results) for results in all_test_results.values())
    total_successful = sum(
        sum(1 for r in results if r.get('success', False))
        for results in all_test_results.values()
    )
    
    # Performance analysis
    all_response_times = []
    for results in all_test_results.values():
        for r in results:
            if 'response_time' in r and r['response_time'] is not None:
                all_response_times.append(r['response_time'])
    
    perf_stats = {
        'min_response_time': min(all_response_times) if all_response_times else 0,
        'max_response_time': max(all_response_times) if all_response_times else 0,
        'avg_response_time': sum(all_response_times) / len(all_response_times) if all_response_times else 0,
        'median_response_time': np.median(all_response_times) if all_response_times else 0,
        'p95_response_time': np.percentile(all_response_times, 95) if all_response_times else 0,
        'p99_response_time': np.percentile(all_response_times, 99) if all_response_times else 0
    }
    
    # Generate detailed report
    report = f"""
# HR Resume Search MCP API - Test Report

**Generated**: {timestamp}  
**API Base URL**: {API_BASE_URL}  
**Test Environment**: {os.getenv('ENVIRONMENT', 'development')}  

## 📊 Executive Summary

| Metric | Value |
|--------|-------|
| Total Tests Executed | {total_tests} |
| Successful Tests | {total_successful} |
| Success Rate | {(total_successful/total_tests*100):.1f}% |
| Average Response Time | {perf_stats['avg_response_time']:.3f}s |
| 95th Percentile Response Time | {perf_stats['p95_response_time']:.3f}s |
| Tests Under 2s Response | {sum(1 for t in all_response_times if t < 2.0)/len(all_response_times)*100:.1f}% |

## 🎯 Test Categories

"""
    
    # Add category details
    for category, results in all_test_results.items():
        if not results:
            continue
            
        successful = sum(1 for r in results if r.get('success', False))
        success_rate = (successful / len(results)) * 100
        
        # Calculate category response times
        cat_response_times = [r['response_time'] for r in results 
                             if 'response_time' in r and r['response_time'] is not None]
        avg_response = sum(cat_response_times) / len(cat_response_times) if cat_response_times else 0
        
        report += f"""
### {category}

- **Tests**: {len(results)}
- **Success Rate**: {success_rate:.1f}%
- **Average Response Time**: {avg_response:.3f}s
- **Status**: {'✅ Passing' if success_rate >= 80 else '⚠️ Needs Attention' if success_rate >= 50 else '❌ Failing'}

"""
        
        # Add failed tests
        failed_tests = [r for r in results if not r.get('success', False)]
        if failed_tests:
            report += "**Failed Tests:**\n"
            for test in failed_tests[:5]:  # Show first 5 failures
                test_name = test.get('test', 'Unknown test')
                status_code = test.get('status_code', 'N/A')
                error = test.get('error', 'No error details')
                report += f"- {test_name}: {status_code} - {error}\n"
            
            if len(failed_tests) > 5:
                report += f"- ... and {len(failed_tests) - 5} more failures\n"
        
        report += "\n"
    
    # Add performance analysis
    report += f"""
## ⚡ Performance Analysis

| Metric | Value |
|--------|-------|
| Minimum Response Time | {perf_stats['min_response_time']:.3f}s |
| Maximum Response Time | {perf_stats['max_response_time']:.3f}s |
| Average Response Time | {perf_stats['avg_response_time']:.3f}s |
| Median Response Time | {perf_stats['median_response_time']:.3f}s |
| 95th Percentile | {perf_stats['p95_response_time']:.3f}s |
| 99th Percentile | {perf_stats['p99_response_time']:.3f}s |

### Performance Recommendations

"""
    
    # Add performance recommendations
    if perf_stats['avg_response_time'] > 2.0:
        report += "❌ **High Response Times**: Average response time exceeds 2s target\n"
    elif perf_stats['avg_response_time'] > 1.0:
        report += "⚠️ **Moderate Response Times**: Consider optimization for better performance\n"
    else:
        report += "✅ **Good Performance**: Response times within acceptable range\n"
    
    if perf_stats['p95_response_time'] > 5.0:
        report += "❌ **Poor P95 Performance**: 95th percentile responses are too slow\n"
    
    # Add client metrics
    report += f"""

## 📈 Client Metrics

| Metric | Value |
|--------|-------|
| Total Requests Made | {client_metrics['requests_made']} |
| Successful Requests | {client_metrics['requests_successful']} |
| Failed Requests | {client_metrics['requests_failed']} |
| Client Success Rate | {client_metrics.get('success_rate', 0):.1f}% |
| Average Response Time | {client_metrics.get('average_response_time', 0):.3f}s |

## 🔍 Detailed Test Results

"""
    
    # Add detailed test results table
    report += "| Test Category | Test Name | Status | Response Time | Status Code |\n"
    report += "|---------------|-----------|--------|---------------|-------------|\n"
    
    for category, results in all_test_results.items():
        for result in results:
            test_name = result.get('test', result.get('name', 'Unknown'))
            status = '✅' if result.get('success', False) else '❌'
            response_time = f"{result.get('response_time', 0):.3f}s"
            status_code = result.get('status_code', 'N/A')
            
            report += f"| {category} | {test_name} | {status} | {response_time} | {status_code} |\n"
    
    # Add recommendations
    report += f"""

## 💡 Recommendations

### Immediate Actions
"""
    
    if total_successful / total_tests < 0.8:
        report += "- 🚨 **Critical**: Success rate below 80% - investigate failing tests immediately\n"
    
    if perf_stats['avg_response_time'] > 2.0:
        report += "- ⚡ **Performance**: Optimize response times - current average exceeds 2s target\n"
    
    # Check for specific issues
    auth_tests = all_test_results.get('Authentication', [])
    auth_success = sum(1 for r in auth_tests if r.get('success', False)) / len(auth_tests) if auth_tests else 1
    
    if auth_success < 0.8:
        report += "- 🔐 **Authentication**: Review authentication endpoints - low success rate\n"
    
    upload_tests = all_test_results.get('Resume Upload', [])
    upload_success = sum(1 for r in upload_tests if r.get('success', False)) / len(upload_tests) if upload_tests else 1
    
    if upload_success < 0.8:
        report += "- 📄 **File Upload**: File upload functionality needs attention\n"
    
    report += f"""

### Long-term Improvements
- 📊 **Monitoring**: Implement continuous performance monitoring
- 🔄 **Automation**: Add these tests to CI/CD pipeline
- 📈 **Metrics**: Set up alerting for performance degradation
- 🧪 **Testing**: Expand test coverage for edge cases

---

**Report Generated by**: HR Resume Search MCP API Testing Suite  
**Notebook**: `notebooks/api_testing.ipynb`  
**Timestamp**: {timestamp}  
"""
    
    return report

# Generate the report
test_report = generate_detailed_report()

# Save report to file
report_file = f"test_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
with open(report_file, 'w') as f:
    f.write(test_report)

print(f"✅ Comprehensive test report saved to: {report_file}")

# Display report summary
display(Markdown("## 📋 Test Report Summary\n\n" + test_report[:1000] + "\n\n*Full report saved to file.*"))

# Clean up
api_client.close()

print("\n🎉 API Testing Suite completed successfully!")
print(f"📊 Total tests executed: {sum(len(results) for results in all_test_results.values())}")
print(f"📝 Report saved: {report_file}")
print(f"🧹 Resources cleaned up")

---

## 🎯 Summary and Next Steps

This comprehensive testing notebook has covered:

### ✅ Completed Tests
1. **API Health Checks** - Verified basic connectivity and endpoint availability
2. **Authentication Flow** - Tested login, token management, and protected endpoints
3. **Resume Upload** - Tested file upload with various formats and Claude AI parsing
4. **Search Functionality** - Comprehensive search testing with different query types
5. **Performance Testing** - Load testing with concurrent requests and stress testing
6. **Data Visualization** - Rich charts and analytics for all test results

### 📊 Key Features
- **Interactive Testing** - Real-time API testing with immediate feedback
- **Performance Metrics** - Detailed response time and success rate analysis
- **Visual Analytics** - Comprehensive charts and dashboards
- **Automated Reporting** - Generated markdown reports for documentation
- **Error Handling** - Robust error detection and reporting

### 🚀 Usage Instructions
1. Ensure API server is running at `http://localhost:8000`
2. Run notebook cells sequentially
3. Monitor test results and visualizations
4. Review generated test report
5. Use insights for API improvement

### 🔧 Customization
- Modify `TEST_TIMEOUT` for different response time requirements
- Adjust `CONCURRENT_REQUESTS` for different load testing scenarios
- Update test data in helper functions for specific testing needs
- Extend visualization functions for custom charts

### 📈 Next Steps
- Integrate with CI/CD pipeline for automated testing
- Add more sophisticated test scenarios
- Implement test data factories for larger datasets
- Create alerts based on performance thresholds

---

**Happy Testing! 🧪✨**